In [44]:
# imports_unificados_completos.py

import os
import pickle
import warnings
import subprocess
import holidays

import numpy as np
import pandas as pd
import shap
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import Image, display

from sklearn import tree
from sklearn.tree import (
    DecisionTreeClassifier, DecisionTreeRegressor,
    export_text, export_graphviz, plot_tree, _tree
)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (
    train_test_split, cross_val_score
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.inspection import permutation_importance

# Funciones personalizadas
from mis_funciones.view_dectree import view_dectree
from mis_funciones.view_dectree_v import view_dectree_v
from mis_funciones.view_detail_text_tree import (
    view_text_tree_detailed, view_text_tree_pruned,
    extract_sorted_tree_paths
)
from mis_funciones.view_detail_text_cattree import (
    view_text_cattree_detailed, view_text_cattree_pruned
)
from mis_funciones.describe_vars import describe_vars
from mis_funciones.describe_vars_txt import describe_vars_txt

In [45]:
# Cargar el dataset
df = pd.read_csv('dataset_suscripciones.csv')


In [ ]:
describe_vars(df)

In [ ]:
sns.boxplot(data=df, x='nivel', y='precio')


In [ ]:


sns.boxplot(data=df, x='nivel', y='precio')
plt.tight_layout()
plt.show()

sns.boxplot(data=df, x='nivel', y='descuento_pct')
plt.tight_layout()
plt.show()

sns.boxplot(data=df, x='nivel', y='rating_promedio')
plt.tight_layout()
plt.show()

sns.boxplot(data=df, x='nivel', y='ventas')
plt.tight_layout()
plt.show()

sns.histplot(data=df, x='ventas', hue='curso', multiple='stack', bins=range(0,10))
plt.tight_layout()
plt.show()

sns.histplot(data=df, x='edad', hue='ventas', multiple='stack', bins=100)
plt.tight_layout()
plt.show()

# Calcular los porcentajes relativos
df_pct = df.groupby(['ventas', 'dispositivo']).size().unstack()
df_pct = df_pct.div(df_pct.sum(axis=1), axis=0) * 100

ax = df_pct.plot(kind='bar', stacked=True)
plt.xlabel('Cantidad de ventas por sesión')
plt.ylabel('Porcentaje')
plt.title('Distribución porcentual de dispositivos por cantidad de ventas')
plt.legend(title='Dispositivo')
plt.tight_layout()
plt.show()

sns.histplot(data=df, x='ventas', hue='referrer', multiple='stack', bins=range(0,10))
plt.tight_layout()
plt.show()

# Calcular los porcentajes relativos
df_pct = df.groupby(['ventas', 'referrer']).size().unstack()
df_pct = df_pct.div(df_pct.sum(axis=1), axis=0) * 100

ax = df_pct.plot(kind='bar', stacked=True)
plt.xlabel('Cantidad de ventas por sesión')
plt.ylabel('Porcentaje')
plt.title('Distribución porcentual de referrer por cantidad de ventas')
plt.legend(title='Referrer')
plt.tight_layout()
plt.show()

tabla = pd.crosstab(df['referrer'], df['ventas'])
tabla_pct = tabla.div(tabla.sum(axis=0), axis=1)
sns.heatmap(tabla_pct, annot=True, fmt=".2f", cmap="YlGnBu", cbar=False)
plt.xlabel('Cantidad de ventas por sesión')
plt.ylabel('Canal de origen (referrer)')
plt.tight_layout()
plt.show()

sns.histplot(data=df, x='ventas', hue='es_domingo', multiple='stack', bins=range(0,10))
plt.tight_layout()
plt.show()

sns.histplot(data=df, x='ventas', hue='es_feriado', multiple='stack', bins=range(0,10))
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df['fecha'] = pd.to_datetime(df['fecha'])
df_diario = df.groupby('fecha').agg({
    'ventas': 'mean',
    'clicks': 'mean',
    'tiempo_browsing': 'mean',
    'paginas_visitadas': 'mean',
    'edad': 'mean',
    'logueado': 'mean',
    'codigo_promocional': 'mean'
}).reset_index()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='ventas')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='clicks')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='tiempo_browsing')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='paginas_visitadas')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='edad')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='logueado')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 2))
sns.lineplot(data=df_diario, x='fecha', y='codigo_promocional')
plt.tight_layout()
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df['fecha'] = pd.to_datetime(df['fecha'])

df_diario = df.groupby('fecha').agg({
    'ventas': 'mean',                    # promedio de ventas por sesión en el día
    'clicks': 'sum',                     # total de clics del día
    'tiempo_browsing': 'sum',            # total de tiempo de navegación del día
    'paginas_visitadas': 'sum',          # total de páginas vistas del día
    'edad': 'mean',                      # edad promedio de los usuarios del día
    'logueado': 'sum',                   # total de sesiones logueadas del día
    'codigo_promocional': 'sum'          # total de usuarios que usaron código ese día
}).reset_index()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='clicks', y='ventas')
plt.xlabel('Total de clics diarios')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='tiempo_browsing', y='ventas')
plt.xlabel('Total de tiempo de navegación (segundos) por día')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='paginas_visitadas', y='ventas')
plt.xlabel('Total de páginas vistas por día')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='edad', y='ventas')
plt.xlabel('Edad promedio de usuarios por día')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='logueado', y='ventas')
plt.xlabel('Cantidad de sesiones logueadas por día')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_diario, x='codigo_promocional', y='ventas')
plt.xlabel('Cantidad de usuarios con código promocional por día')
plt.ylabel('Promedio de ventas por sesión')
plt.tight_layout()
plt.show()

In [ ]:
describe_vars(df)

# Análisis de relaciones
## Regresiones


In [ ]:


# --- REGRESIÓN BASE ---
df_reg_base = pd.DataFrame({
    'Ventas': df['ventas'].astype(float),
    'Clicks': df['clicks'].astype(float),
    'Tiempo_Browsing': df['tiempo_browsing'].astype(float),
    'Paginas_Visitadas': df['paginas_visitadas'].astype(float),
    'Logueado': df['logueado'].astype(int),
    'Codigo_Promocional': df['codigo_promocional'].astype(int),
    'Es_Feriado': df['es_feriado'].astype(int),
    'Es_Domingo': df['es_domingo'].astype(int),
    'Dia_Semana': df['dia_semana'].astype(int),
    'Hora': df['hora'].astype(int),
    'Dias_Desde_Promocion': df['dias_desde_promocion'].astype(int)
})
X_base = sm.add_constant(df_reg_base.drop(columns=['Ventas']))
modelo_base = sm.OLS(df_reg_base['Ventas'], X_base).fit()

print("\n=== REGRESIÓN BASE ===\n")
print(modelo_base.summary())


In [ ]:

# --- REGRESIÓN EXTENDIDA ---
df_reg_ext = pd.DataFrame({
    'Ventas': df['ventas'].astype(float),
    'Clicks': df['clicks'].astype(float),
    'Tiempo_Browsing': df['tiempo_browsing'].astype(float),
    'Paginas_Visitadas': df['paginas_visitadas'].astype(float),
    'Precio': df['precio'].astype(float),
    'Descuento': df['descuento_pct'].astype(float),
    'Rating': df['rating_promedio'].astype(float),
    'Edad': df['edad'].astype(float),
    'Es_Feriado': df['es_feriado'].astype(int),
    'Es_Domingo': df['es_domingo'].astype(int),
    'Dia_Semana': df['dia_semana'].astype(int),
    'Hora': df['hora'].astype(int),
    'Dias_Desde_Promocion': df['dias_desde_promocion'].astype(int),
    'Logueado': df['logueado'].astype(int),
    'Codigo_Promocional': df['codigo_promocional'].astype(int)
})
cat_vars = ['curso', 'nivel', 'dispositivo', 'referrer', 'nivel_educativo']
df_dummies = pd.get_dummies(df[cat_vars], drop_first=True).astype(float)
df_reg_ext = pd.concat([df_reg_ext, df_dummies], axis=1)
X_ext = sm.add_constant(df_reg_ext.drop(columns=['Ventas']))
modelo_ext = sm.OLS(df_reg_ext['Ventas'], X_ext).fit()

print("\n=== REGRESIÓN EXTENDIDA ===\n")
print(modelo_ext.summary())


In [ ]:

# --- COMPARACIÓN ---
def marcar(valor, p):
    if p < 0.01:
        return f"{valor:.4f}***"
    elif p < 0.05:
        return f"{valor:.4f}** "
    elif p < 0.1:
        return f"{valor:.4f}*  "
    else:
        return f"{valor:.4f}   "

param_base = pd.Series({k: marcar(v, modelo_base.pvalues[k]) for k, v in modelo_base.params.items()}, name="Regresión Base")
param_ext = pd.Series({k: marcar(v, modelo_ext.pvalues[k]) for k, v in modelo_ext.params.items()}, name="Regresión Extendida")
comparacion = pd.concat([param_base, param_ext], axis=1)

# Agregar R² al final de la tabla
r2_row = pd.DataFrame({
    'Regresión Base': [f"R² = {modelo_base.rsquared:.4f}"],
    'Regresión Extendida': [f"R² = {modelo_ext.rsquared:.4f}"]
}, index=[" "])

comparacion = pd.concat([comparacion, r2_row])

print("\n=== COMPARACIÓN DE REGRESORES Y SIGNIFICACIÓN ===\n")
print(comparacion.fillna("").to_string())

In [ ]:
#hay una relación lineal entre paginas visitadas y ventas, esto se captura bien en las regresiones

mean_std = df.groupby('paginas_visitadas', observed=True)['ventas'].agg(['mean', 'std', 'count']).reset_index()
mean_std['se'] = mean_std['std'] / mean_std['count']**0.5  # error estándar
plt.figure(figsize=(8, 5))
sns.barplot(data=mean_std, x='paginas_visitadas', y='mean', color='steelblue', errorbar=None, alpha=0.6, linewidth=1, edgecolor='darkblue')
plt.errorbar(x=mean_std['paginas_visitadas'], y=mean_std['mean'], yerr=mean_std['se'],
            fmt='none', ecolor='lightgray', capsize=3, linewidth=1)
z_curva = np.polyfit(mean_std['paginas_visitadas'], mean_std['mean'], 3)
p_curva = np.poly1d(z_curva)
plt.plot(mean_std['paginas_visitadas'], p_curva(mean_std['paginas_visitadas']), color='red', linestyle='--', linewidth=1.5)
z_lineal = np.polyfit(mean_std['paginas_visitadas'], mean_std['mean'], 1)
p_lineal = np.poly1d(z_lineal)
plt.plot(mean_std['paginas_visitadas'], p_lineal(mean_std['paginas_visitadas']), color='gray', linestyle='-', linewidth=1.2)
plt.fill_between(mean_std['paginas_visitadas'], p_lineal(mean_std['paginas_visitadas']) - mean_std['se'], 
                p_lineal(mean_std['paginas_visitadas']) + mean_std['se'], color='gray', alpha=0.5)
plt.title("Promedio de Ventas por Páginas Visitadas")
plt.xlabel("Páginas Visitadas")
plt.ylabel("Promedio de Ventas")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# No hay una relación clara entre precio y ventas, por lo que la aproximación lineal es insuficiente. Vamos a necesitar otro modelo.

plt.figure(figsize=(8, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(df)))
scatter = plt.scatter(df['precio'], df['ventas'], 
                    c=df['precio'],
                    cmap='inferno_r', # Alternatives: plasma, magma, inferno, cividis
                    alpha=0.5)
plt.colorbar(scatter, label='Precio')
plt.title("Relación no lineal: Precio vs Ventas")
plt.xlabel("Precio")
plt.ylabel("Ventas")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# En efecto... PRECIO no da ningun efecto...y claramente su relación con ventas es no lineal. 
# Precio -->   beta:  -0.0002    p-value:   0.950 

# veamos nuevamente la regresión extendida...
df_reg_ext = pd.DataFrame({
    'Ventas': df['ventas'].astype(float),
    'Clicks': df['clicks'].astype(float),
    'Tiempo_Browsing': df['tiempo_browsing'].astype(float),
    'Paginas_Visitadas': df['paginas_visitadas'].astype(float),
    'Precio': df['precio'].astype(float),
    'Descuento': df['descuento_pct'].astype(float),
    'Rating': df['rating_promedio'].astype(float),
    'Edad': df['edad'].astype(float),
    'Es_Feriado': df['es_feriado'].astype(int),
    'Es_Domingo': df['es_domingo'].astype(int),
    'Dia_Semana': df['dia_semana'].astype(int),
    'Hora': df['hora'].astype(int),
    'Dias_Desde_Promocion': df['dias_desde_promocion'].astype(int),
    'Logueado': df['logueado'].astype(int),
    'Codigo_Promocional': df['codigo_promocional'].astype(int)
})
cat_vars = ['curso', 'nivel', 'dispositivo', 'referrer', 'nivel_educativo']
df_dummies = pd.get_dummies(df[cat_vars], drop_first=True).astype(float)
df_reg_ext = pd.concat([df_reg_ext, df_dummies], axis=1)
X_ext = sm.add_constant(df_reg_ext.drop(columns=['Ventas']))
modelo_ext = sm.OLS(df_reg_ext['Ventas'], X_ext).fit()

print("\n=== REGRESIÓN EXTENDIDA ===\n")
print(modelo_ext.summary())


---

# Decision trees, round 2


# 
![](./model_outputs/output.png)


### ¿Qué es un Árbol de Regresión?
- Es un algoritmo que predice valores numéricos (continuos) mediante divisiones sucesivas del espacio de datos.
- A cada división se le llama split y ocurre sobre una variable que minimiza el error de predicción (MSE).
- El resultado es una estructura de árbol con reglas condicionales.

---

### Ejemplo mínimo

##### Evaluación de todos los cortes posibles en `precio` para explicar `ventas`

Usamos el siguiente dataset:

| precio | ventas |
|--------|--------|
| 10     | 6      |
| 15     | 5      |
| 20     | 3      |
| 25     | 2      |
| 30     | 1      |

Posibles cortes entre valores únicos ordenados:

| Cortes candidatos | Valor |
|------------------|-------|
| (10+15)/2        | 12.5  |
| (15+20)/2        | 17.5  |
| (20+25)/2        | 22.5  |
| (25+30)/2        | 27.5  |


---

Calculamos el MSE para corte o grupo, a ver cuál es mejor (menos error MSE, mejor): 

$$
MSE = \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{n}
$$


---

##### Corte 1: precio ≤ 12.5

- Grupo 1: [10] → media = 6  
- Grupo 2: [15, 20, 25, 30] → media = 2.75  

MSE = $\frac{(6-6)^2 + (5-2.75)^2 + (3-2.75)^2 + (2-2.75)^2 + (1-2.75)^2}{5}$  
MSE = $\frac{0 + 5.06 + 0.06 + 0.56 + 3.06}{5} = \mathbf{1.75}$

| precio | ventas | grupo 1 (μ=6) | grupo 2 (μ=2.75) | MSE Corte 1 |
|--------|--------|----------------|------------------|--------------|
| 10     | 6      | x              |                  | 1.75         |
| 15     | 5      |                | x                |              |
| 20     | 3      |                | x                |              |
| 25     | 2      |                | x                |              |
| 30     | 1      |                | x                |              |

---

##### Corte 2: precio ≤ 17.5

- Grupo 1: [10, 15] → media = 5.5  
- Grupo 2: [20, 25, 30] → media = 2.0  

MSE = $\frac{(6-5.5)^2 + (5-5.5)^2 + (3-2)^2 + (2-2)^2 + (1-2)^2}{5}$  
MSE = $\frac{0.25 + 0.25 + 1 + 0 + 1}{5} = \mathbf{0.5}$

| precio | ventas | grupo 1 (μ=5.5) | grupo 2 (μ=2.0) | MSE Corte 2 |
|--------|--------|------------------|-----------------|-------------|
| 10     | 6      | x                |                 | 0.5         |
| 15     | 5      | x                |                 |             |
| 20     | 3      |                  | x               |             |
| 25     | 2      |                  | x               |             |
| 30     | 1      |                  | x               |             |

---

##### Corte 3: precio ≤ 22.5

- Grupo 1: [10, 15, 20] → media = 4.67  
- Grupo 2: [25, 30] → media = 1.5  

MSE = $\frac{(6-4.67)^2 + (5-4.67)^2 + (3-4.67)^2 + (2-1.5)^2 + (1-1.5)^2}{5}$  
MSE = $\frac{1.76 + 0.11 + 2.78 + 0.25 + 0.25}{5} = \mathbf{1.03}$

| precio | ventas | grupo 1 (μ=4.67) | grupo 2 (μ=1.5) | MSE Corte 3 |
|--------|--------|------------------|------------------|--------------|
| 10     | 6      | x                |                  | 1.03         |
| 15     | 5      | x                |                  |              |
| 20     | 3      | x                |                  |              |
| 25     | 2      |                  | x                |              |
| 30     | 1      |                  | x                |              |

---

##### Corte 4: precio ≤ 27.5

- Grupo 1: [10, 15, 20, 25] → media = 4  
- Grupo 2: [30] → media = 1  

MSE = $\frac{(6-4)^2 + (5-4)^2 + (3-4)^2 + (2-4)^2 + (1-1)^2}{5}$  
MSE = $\frac{4 + 1 + 1 + 4 + 0}{5} = \mathbf{2.0}$

| precio | ventas | grupo 1 (μ=4) | grupo 2 (μ=1) | MSE Corte 4 |
|--------|--------|---------------|---------------|-------------|
| 10     | 6      | x             |               | 2.0         |
| 15     | 5      | x             |               |             |
| 20     | 3      | x             |               |             |
| 25     | 2      | x             |               |             |
| 30     | 1      |               | x             |             |

---

### Resultado:

| Corte         | MSE   |
|---------------|-------|
| precio ≤ 12.5 | 1.75  |
| precio ≤ 17.5 | 0.50  |
| precio ≤ 22.5 | 1.03  |
| precio ≤ 27.5 | 2.00  |

Comparamos todos los cortes: 

> #### **Corte óptimo**: `precio ≤ 17.5`, con MSE = **0.50**


# Entonces, ¿Qué hace el árbol de regresión?

- Construye reglas jerárquicas que dividen el conjunto de datos según umbrales óptimos.
- Por ejemplo, con el dataset:

  | precio | ventas |
  |--------|--------|
  | 10     | 6      |
  | 15     | 5      |
  | 20     | 3      |
  | 25     | 2      |
  | 30     | 1      |

Comparando posibles cortes sobre `precio`, el árbol evalúa los valores intermedios entre observaciones ordenadas:
- Corte óptimo: `precio ≤ 17.5` → MSE total = **0.50**, el menor entre todos los candidatos.

  El árbol puede construir reglas como:

>  - Si `precio ≤ 17.5` → predicción ventas = 5.5  
>  - Si `precio > 17.5` → predicción ventas = 2.0

Cada regla define un nodo hoja (`leaf node`) en el que la predicción es el promedio de las observaciones en ese grupo.



---
# Calculemos el decision tree para la regresión anterior, a ver qué pasa con ventas! 
![](./model_outputs/reg_ventas.png)


In [ ]:


# Features y target
X = df[[
    'clicks', 'tiempo_browsing', 'paginas_visitadas', 'precio',
    'descuento_pct', 'rating_promedio', 'edad', 'es_feriado', 'es_domingo',
    'dia_semana', 'hora', 'dias_desde_promocion', 'logueado',
    'codigo_promocional'
]]
y = df['ventas']

# División en entrenamiento y testeo (80/20) >>> test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Árbol de regresión
tree = DecisionTreeRegressor(max_depth=4, criterion='squared_error', random_state=42)
tree.fit(X_train, y_train)

# MSE
y_pred_train = tree.predict(X_train)
y_pred_test = tree.predict(X_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"\n=== DECISION TREE REGRESIÓN ===")
print(f"MSE entrenamiento: {mse_train:.4f}")
print(f"MSE testeo:        {mse_test:.4f}")

# Visualización del árbol
view_dectree(
    tree_model=tree, 
    features=list(X.columns),
    nombre_base='arbol_regresion_ventas', 
    horizontal_spacing=1.2,
    vertical_spacing=1.2,
    font_size=16,
    flecha_grosor_factor=2,
    min_leaf_pct=0.01
)



In [ ]:


print("\n=== ÁRBOL DE DECISIÓN CON DETALLES ===\n")
print(view_text_tree_detailed(tree, list(X.columns)))

In [ ]:
print("\n=== PRUNED: ÁRBOL DE DECISIÓN CON DETALLES, sin nodos con menos del 5% de muestra ===\n")
print(view_text_tree_pruned(tree, list(X.columns)))

In [ ]:
print(extract_sorted_tree_paths(tree, list(X.columns)))

# Procedimiento para interpretar el árbol de decisión respecto del precio


## 1. Identificar las hojas con mayor valor esperado de ventas
Localizar los nodos terminales (leaves) con mayores valores de value (predicción de ventas). Estos representan combinaciones de condiciones que conducen a mayor desempeño.

## 2. Reconstruir el camino de decisión hacia esas hojas  
Desde cada hoja seleccionada, recorrer hacia atrás el árbol hasta la raíz, recuperando las reglas (splits) que determinan esas predicciones.

## 3. Filtrar en esos caminos los nodos donde aparece precio
Enfocarse sólo en las ramas donde la variable precio participa activamente como criterio de división.

## 4. Extraer los rangos de precios asociados a mayores ventas
A partir de los cortes de precio en esos caminos, derivar los intervalos de precio que están vinculados con mayores ventas.

## 5. Analizar las interacciones entre precio y otras variables en esas ramas
Observar qué otras variables acompañan al precio en esas rutas (por ejemplo: paginas_visitadas, logueado, descuento_pct, etc.), y cómo la combinación de esos factores modifica el nivel de ventas.

---

Este análisis permite:

- Inferir condiciones específicas de precio bajo las cuales las ventas aumentan
- Detectar interacciones relevantes que un modelo lineal probablemente no captaría



### Análisis del árbol de decisión respecto a la variable `precio`

(sólo hojas con N ≥ 0.5% del total)

#### Paso 1: Hojas con mayor valor de ventas

Se identifican las siguientes hojas con valores predichos más altos y soporte muestral:

- 1.86 → paginas_visitadas > 5.5, precio > 19.28, hora > 1.5
- 1.60 → paginas_visitadas > 5.5, logueado > 0.5, descuento_pct ≤ 38.25  
- 1.58 → paginas_visitadas ≤ 5.5, logueado > 0.5, dia_semana > 4.5, precio > 25.05
- 1.55 → paginas_visitadas > 5.5, logueado ≤ 0.5, edad > 37.33
- 1.34 → paginas_visitadas ≤ 5.5, logueado > 0.5, dia_semana ≤ 4.5, precio > 17.02
- 1.33 → paginas_visitadas > 5.5, logueado ≤ 0.5, edad ≤ 37.33
- 1.26 → paginas_visitadas ≤ 5.5, logueado ≤ 0.5, rating_promedio > 2.53, precio > 13.09
- 1.04 → paginas_visitadas ≤ 5.5, logueado > 0.5, dia_semana > 4.5, precio ≤ 25.05


#### Paso 2: Presencia de `precio` como criterio de decisión

La variable `precio` aparece como nodo de decisión en tres contextos estructurales del árbol:

1. **Usuarios no logueados con pocas páginas visitadas** (`paginas_visitadas ≤ 5.5`, `logueado = 0`):  
   - Si además tienen un `rating_promedio > 2.53`, se bifurca por `precio`.  
     • Es decir, el modelo considera que, para estos usuarios, la combinación de nivel medio-alto de rating y no estar logueado requiere discriminar según `precio` para estimar las ventas.  
   - En este subgrupo (rama):  
     - si `precio > 13.09`, está asociado a ventas de **1.26**.  
     - si `precio ≤ 13.09`, el valor predicho es **5.00**, pero esa hoja tiene **N = 1**, por lo que se considera no representativa.

2. **Usuarios logueados con pocas páginas visitadas** (`paginas_visitadas ≤ 5.5`, `logueado = 1`):  
   - En este segmento, el árbol primero distingue si es un día de semana (`dia_semana ≤ 4.5`) o fin de semana (`dia_semana > 4.5`).  
     • Es decir, el modelo introduce el día como variable de contexto para determinar el rol del `precio` en la predicción.  
   - En este subgrupo (rama):  
     - en días hábiles, si `precio > 17.02`, se predicen ventas de **1.34**.  
     - en fines de semana, si `precio > 25.05`, se predicen ventas de **1.58**; si `precio ≤ 25.05`, se predicen ventas de **1.04**.

3. **Usuarios con muchas páginas visitadas** (`paginas_visitadas > 9.5`):  
   - En este segmento, `precio` es el primer nodo de decisión.  
     • Es decir, el modelo interpreta que, cuando un usuario navega muchas páginas, el valor del `precio` es el factor más relevante para estimar la probabilidad de compra.  
   - En este subgrupo (rama):  
     - si `precio > 19.28` y `hora > 1.5`, se predicen ventas de **1.86**, uno de los valores más altos del árbol con N significativo.  
     - si `precio > 19.28` y `hora ≤ 1.5`, se predicen ventas de **2.64**, pero con N bajo (0.3%), por lo que se considera marginal.  
     - si `precio ≤ 19.28`, el árbol bifurca según `descuento_pct`, pero las hojas con mayor predicción en este tramo (por ejemplo, **4.50**) tienen N < 0.5%, por lo que no se consideran confiables.

### Paso 3: Rango de precios asociado a mayores ventas (con N ≥ 0.5%)

- Cuando `precio > 19.28` y además `hora > 1.5`, se predicen ventas de **1.86**, el valor más alto con soporte muestral válido en todo el árbol.
- En el segmento de usuarios logueados en días de semana, `precio > 17.02` está asociado a **1.34** ventas, mientras que valores menores o iguales conducen a hojas con N muy bajo (por ejemplo, 3.50 ventas con N = 4).
- En el mismo grupo de usuarios logueados, pero en fines de semana, `precio > 25.05` se asocia a un leve incremento de ventas, alcanzando **1.58**, mientras que precios menores o iguales se relacionan con **1.04**.

### Paso 4: Conjunciones más efectivas

- `(precio > 17.02) AND (dia_semana ≤ 4.5)` → ventas de **1.34** en usuarios logueados entre semana; se trata de una hoja con alta representatividad (N = 2107).
- `(precio > 25.05) AND (dia_semana > 4.5)` → ventas de **1.58** en usuarios logueados durante fines de semana.
- `(precio > 13.09) AND (rating_promedio > 2.53)` → ventas de **1.26** en usuarios no logueados con pocas páginas y valoración media-alta; esta combinación define una subrama con N = 1970.


---

## ¿Predicción más alta o respaldo muestral?

Cuando analizamos un árbol de decisión de regresión, muchas veces aparecen ramas que predicen valores altos pero con muy pocos casos (`N`), y otras que predicen valores más bajos pero con gran respaldo empírico. ¿Cuál debería guiar nuestras decisiones?

#### Ejemplo

- **Predicted Value: 1.86**  
  N: 255 casos (3.2%)  
  → Alta predicción, bajo respaldo

- **Predicted Value: 1.60**  
  N: 1661 casos (20.8%)  
  → Predicción más baja, alto respaldo

#### ¿Cuál es mejor?

Todo depende del objetivo de la recomendación. Hay tres formas posibles de evaluar cuál “domina”:

---
#### A. Maximizar la **predicción individual**

> Se elige el valor más alto, sin importar el N.

Este criterio sirve si el interés es identificar **casos con máximo potencial unitario**, aunque sean raros. Se recomienda con cautela porque el respaldo empírico puede ser débil.

**Dominante**: Predicted Value = 1.86

---

#### B. Para maximizar el **volumen total esperado**

> Se multiplica `Predicted × N` para obtener una estimación del impacto total.

Calculamos:

- 1.86 × 255 ≈ **474 unidades esperadas**
- 1.60 × 1661 ≈ **2658 unidades esperadas**

Aunque el valor predicho es menor, la segunda opción tiene mayor impacto acumulado por el tamaño del grupo.

**Dominante**: Predicted Value = 1.60

---

#### C. Para obtener una recomendación **robusta y generalizable**

> Se utiliza un umbral mínimo de N (por ejemplo, 0.5% del total)  
> y dentro de ese conjunto, se ordenan por valor predicho.

Esto evita tomar decisiones sobre combinaciones que casi no aparecen en los datos, y asegura confiabilidad estadística. Es el criterio más conservador y recomendado para automatizar decisiones.

**Dominante**: la opción con mayor `predicted` entre las hojas con N ≥ 0.5%

---


## Recomendaciones de precios según comportamiento del usuario

A partir del análisis del árbol de decisión entrenado sobre los datos de ventas, se identificaron patrones que combinan comportamiento de navegación, estado del usuario y precios. Los resultados muestran que las decisiones óptimas de precio dependen del perfil de usuario, la intensidad de su navegación y el contexto temporal. A continuación, se sistematizan los escenarios más representativos en función de esas combinaciones.

---

### Escenarios representativos con ventas altas (ventas ≥ 1.86, N ≥ 0.5%)

#### 1. Precio alto en usuarios altamente comprometidos
- **Condiciones**:
  - `paginas_visitadas > 5.5`
  - `paginas_visitadas > 9.5`
  - `precio > 19.28`
  - `hora > 1.50`
- **Ventas esperadas**: 1.86
- **Relevancia muestral**: N = 255 (3.2%)
- **Interpretación**: en usuarios muy activos, el precio deja de ser una barrera si el horario es fuera de la madrugada. Las ventas se mantienen incluso con precios relativamente altos.

#### 2. Descuentos intermedios en usuarios comprometidos
- **Condiciones**:
  - `paginas_visitadas > 5.5`
  - `paginas_visitadas <= 9.5`
  - `logueado == 1`
  - `descuento_pct <= 38.25`
- **Ventas esperadas**: 1.60
- **Relevancia muestral**: N = 1661 (20.8%)
- **Interpretación**: los descuentos no necesitan ser extremos. En usuarios activos y logueados, el descuento medio genera un efecto claro de conversión.

#### 3. Precio alto en fines de semana logueados
- **Condiciones**:
  - `paginas_visitadas <= 5.5`
  - `logueado == 1`
  - `dia_semana > 4.5`
  - `precio > 25.05`
- **Ventas esperadas**: 1.58
- **Relevancia muestral**: N = 776 (9.7%)
- **Interpretación**: durante fines de semana, los usuarios logueados pueden tolerar precios altos. La conversión se mantiene en niveles aceptables si hay fuerte presencia.

#### 4. Adultos no logueados con navegación intermedia
- **Condiciones**:
  - `paginas_visitadas > 5.5`
  - `paginas_visitadas <= 9.5`
  - `logueado == 0`
  - `edad > 37.33`
- **Ventas esperadas**: 1.55
- **Relevancia muestral**: N = 343 (4.3%)
- **Interpretación**: los adultos que no se loguean pero navegan bastante terminan comprando en niveles moderadamente altos. El precio no aparece explícitamente, lo que sugiere que no es barrera en este subgrupo.

#### 5. Precio medio para logueados en semana
- **Condiciones**:
  - `paginas_visitadas <= 5.5`
  - `logueado == 1`
  - `dia_semana <= 4.5`
  - `precio > 17.02`
- **Ventas esperadas**: 1.34
- **Relevancia muestral**: N = 2107 (26.3%)
- **Interpretación**: incluso con precios medios-altos, los usuarios logueados entre semana muestran buena tasa de conversión. Este es el segmento más estable y representativo.

---

### Conclusiones para política de precios

- Las combinaciones más efectivas se concentran en usuarios activos, con navegación amplia o logueo.
- El precio es un factor relevante, pero actúa de forma contextual: puede elevarse si otras variables indican alto engagement.
- Los mejores resultados aparecen con precios moderados o intermedios (17 a 25) más que con mínimos.
- Se sugiere aplicar **estrategias dinámicas de precios**, donde el modelo detecte señales de compromiso en tiempo real (páginas vistas, horario, tipo de usuario) y ajuste el precio ofertado para maximizar ventas sin necesidad de descuentos extremos.

## Recomendaciones de precios según comportamiento del usuario

A partir del análisis del árbol de decisión entrenado sobre los datos de ventas, se identificaron patrones que combinan comportamiento de navegación, estado del usuario y precios. Los resultados muestran que las decisiones óptimas de precio dependen del perfil de usuario, la intensidad de su navegación y el contexto temporal. A continuación, se sistematizan los escenarios que presentan los valores de ventas más altos bajo distintas combinaciones de condiciones.

---

### Escenarios con valores predichos de ventas elevados (≥ 3.5)

#### 1. Rating bajo, no logueado, pocas páginas
- **Condiciones**:
  - `paginas_visitadas ≤ 5.5`
  - `logueado == 0`
  - `rating_promedio ≤ 2.53`
  - `clicks ≤ 6.5`
- **Precio considerado**: no interviene
- **Ventas predichas**: 6.00
- **Observación**: N=1 (0.0%) → no representativo

#### 2. Rating alto, no logueado, precio bajo
- **Condiciones**:
  - `paginas_visitadas ≤ 5.5`
  - `logueado == 0`
  - `rating_promedio > 2.53`
  - `precio ≤ 13.09`
- **Ventas predichas**: 5.00
- **Observación**: N=1 (0.0%) → no representativo

#### 3. Usuario logueado en días hábiles, precio moderado
- **Condiciones**:
  - `paginas_visitadas ≤ 5.5`
  - `logueado == 1`
  - `dia_semana ≤ 4.5`
  - `precio ≤ 17.02`
- **Ventas predichas**: 3.50
- **Observación**: N=4 (0.1%) → marginalmente representativo

#### 4. Descuentos agresivos con navegación intermedia
- **Condiciones**:
  - `5.5 < paginas_visitadas ≤ 9.5`
  - `logueado == 1`
  - `descuento_pct > 38.25`
- **Ventas predichas**: 3.60
- **Observación**: N=5 (0.1%) → marginalmente representativo

#### 5. Navegación intensiva con descuento moderado
- **Condiciones**:
  - `paginas_visitadas > 9.5`
  - `precio ≤ 19.28`
  - `descuento_pct > 20.15`
- **Ventas predichas**: 4.50
- **Observación**: N=2 (0.0%) → no representativo

---

### Política de precios

- Las predicciones más altas de ventas ocurren bajo condiciones particulares de navegación, estado del usuario y descuentos, y no pueden atribuirse de forma directa al precio de forma aislada.
- El **precio no es determinante lineal ni autónomo**: su influencia depende del contexto y del perfil del usuario. 
- **Valores altos de precio (superiores a 25)** no están asociados a mayores niveles de ventas, incluso cuando el usuario está logueado o en días no laborables.
- Se recomienda implementar **estrategias de precios diferenciadas**, ajustadas a señales de comportamiento como navegación activa, estado de sesión (logueado o no), nivel de rating promedio, y momento temporal (día de la semana, hora).
- Las hojas con mayor valor predicho no tienen respaldo empírico suficiente (N < 0.5%), por lo que **no deben ser consideradas como evidencia sólida**, aunque pueden guiar hipótesis de testeo futuro.

# Siguiendo la misma lógica, podemos hacer la interpretación general del árbol

### Interpretación general del árbol de decisión

El árbol de decisión genera reglas jerárquicas para predecir el número de ventas (`ventas`) mediante divisiones sucesivas basadas en variables explicativas. Cada nodo intermedio representa una condición binaria sobre una variable, y cada nodo hoja indica una predicción promedio de ventas para un subconjunto específico de usuarios. La variable `paginas_visitadas` funciona como nodo raíz, dividiendo el árbol en dos grandes ramas: usuarios de baja navegación (≤ 5.5) y usuarios con navegación más intensiva (> 5.5).

---

### Subárbol izquierdo: usuarios con navegación baja (paginas_visitadas ≤ 5.5)

#### Usuarios no logueados (`logueado ≤ 0.5`)
- Si `rating_promedio ≤ 2.53`:
  - Si `clicks ≤ 6.5`: **ventas = 6.00**, pero con N=1 → no representativo
  - Si `clicks > 6.5`: **ventas = 2.00**, también con N=1 → no representativo
- Si `rating_promedio > 2.53`:
  - Si `precio ≤ 13.09`: **ventas = 5.00**, N=1 → no representativo
  - Si `precio > 13.09`: **ventas = 1.26**, N=1970 (24.6%) → representativo

#### Usuarios logueados (`logueado > 0.5`)
- Si `dia_semana ≤ 4.5` (días hábiles):
  - Si `precio ≤ 17.02`: **ventas = 3.50**, N=4 (0.1%) → marginalmente representativo
  - Si `precio > 17.02`: **ventas = 1.34**, N=2107 (26.3%) → representativo
- Si `dia_semana > 4.5` (fin de semana):
  - Si `precio ≤ 25.05`: **ventas = 1.04**, N=67 (0.8%)
  - Si `precio > 25.05`: **ventas = 1.58**, N=776 (9.7%)

---

### Subárbol derecho: usuarios con navegación media o alta (paginas_visitadas > 5.5)

#### Visitas medias (paginas_visitadas ≤ 9.5)

- Si `logueado ≤ 0.5`:
  - Si `edad ≤ 37.33`: **ventas = 1.33**, N=782 (9.8%)
  - Si `edad > 37.33`: **ventas = 1.55**, N=343 (4.3%)

- Si `logueado > 0.5`:
  - Si `descuento_pct ≤ 38.25`: **ventas = 1.60**, N=1661 (20.8%)
  - Si `descuento_pct > 38.25`: **ventas = 3.60**, N=5 (0.1%) → marginalmente representativo

#### Visitas intensas (paginas_visitadas > 9.5)

- Si `precio ≤ 19.28`:
  - Si `descuento_pct ≤ 20.15`: **ventas = 3.00**, N=3 → no representativo
  - Si `descuento_pct > 20.15`: **ventas = 4.50**, N=2 → no representativo

- Si `precio > 19.28`:
  - Si `hora ≤ 1.50`: **ventas = 2.64**, N=22 (0.3%)
  - Si `hora > 1.50`: **ventas = 1.86**, N=255 (3.2%)

---

### Conclusiones clave

- Las predicciones más altas del árbol se producen en subgrupos con baja evidencia empírica (N ≤ 1), lo que limita su valor para recomendaciones prácticas.
- Entre los escenarios representativos (N ≥ 0.5%), los valores de ventas se concentran entre 1.04 y 1.86.
- El precio actúa de manera **condicional**: no es un factor determinante por sí mismo, sino que su efecto depende de otras variables como el rating, el logueo o el día de la semana.
- El árbol identifica configuraciones no lineales y no aditivas: por ejemplo, el mismo precio puede ser favorable o desfavorable según la hora o el tipo de usuario.
- La estructura permite captar perfiles de comportamiento diferenciados (ej. logueados entre semana, no logueados con buen rating, usuarios intensivos con descuentos), lo cual excede las capacidades de un modelo lineal estándar.

| Variable                    | Regresión Temporal | Regresión Extendida | Árbol de Decisión                         | Interacción estructural en árbol                                   |
|----------------------------|--------------------|---------------------|-------------------------------------------|--------------------------------------------------------------------|
| **Intercepto**             | 0.9390***          | 0.5838**            | -                                         | -                                                                  |
| Clicks                     | -0.0095            | -0.0098             | Condición terminal (nivel 3, N bajo)      | en usuarios no logueados con navegación baja y rating bajo         |
| Tiempo_Browsing            | 0.0005**           | 0.0005**            | Condición intermedia (nivel 3, N bajo)    | en usuarios no logueados con navegación baja y rating bajo         |
| Paginas_Visitadas          | 0.0556***          | 0.0551***           | Variable de segmentación raíz (nivel 0)   | todas las ramas (nivel raíz)                                       |
| Feriado                    | 0.0164             | 0.0180              | -                                         | -                                                                  |
| Domingo                    | 0.1659***          | 0.1609***           | Condición intermedia (nivel 2)            | en usuarios logueados con navegación baja                          |
| Dia_Semana                 | 0.0046             | 0.0057              | Condición intermedia (nivel 2)            | en usuarios logueados con navegación baja                          |
| Hora                       | 0.0039**           | 0.0038**            | Condición terminal (nivel 4)              | en usuarios con navegación alta y precios altos                    |
| Dias_Desde_Promocion       | 0.0020             | 0.0019              | -                                         | -                                                                  |
| Logueado                   | 0.1213***          | 0.1229***           | Condición estructurante (nivel 1)         | divide navegación baja y navegación media/alta                     |
| Codigo_Promocional         | 0.1673***          | 0.1637***           | Condición terminal (nivel 4)              | en usuarios logueados con navegación media                         |
| Precio                     | —                  | -0.0002             | Condición intermedia (niveles 2–3)        | aparece en múltiples ramas: rating alto, logueados, usuarios activos |
| Descuento                  | —                  | -0.0022             | Condición intermedia (niveles 2–3)        | en usuarios con navegación media y alta                            |
| Rating                     | —                  | 0.0989**            | Condición estructurante (nivel 1)         | en usuarios no logueados con navegación baja                       |
| Edad                       | —                  | -0.0013             | Condición terminal (nivel 4)              | en usuarios no logueados con navegación media                      |
| curso_Finanzas             | —                  | -0.0463             | -                                         | -                                                                  |
| curso_Liderazgo            | —                  | -0.0093             | -                                         | -                                                                  |
| nivel_Inicial              | —                  | 0.1062              | -                                         | -                                                                  |
| nivel_Intermedio           | —                  | 0.0576              | -                                         | -                                                                  |
| dispositivo_PC             | —                  | 0.1548***           | -                                         | -                                                                  |
| referrer_competencia       | —                  | -0.0222             | -                                         | -                                                                  |
| referrer_correo            | —                  | 0.0858**            | -                                         | -                                                                  |
| referrer_otros             | —                  | -0.0090             | -                                         | -                                                                  |
| referrer_redes             | —                  | 0.0180              | -                                         | -                                                                  |
| referrer_youtube           | —                  | 0.0918**            | -                                         | -                                                                  |
| nivel_educativo_Secundario| —                  | -0.1216***          | -                                         | -                                                                  |
| nivel_educativo_Terciario | —                  | -0.1515***          | -                                         | -                                                                  |
| nivel_educativo_Universit.| —                  | -0.0577             | -                                         | -                                                                  |

---

### Notas sobre el árbol de decisión:

- **Clicks** y **Tiempo_Browsing**: sólo aparecen como nodos finales en una rama poco representativa (usuarios no logueados, baja navegación, rating bajo).
- **Paginas_Visitadas**: define la raíz y estructura principal del árbol; divide entre baja, media y alta navegación.
- **Logueado**: actúa como variable clave que segmenta comportamiento y sensibilidad al precio y descuentos.
- **Precio**: relevante pero no determinante por sí solo. Su impacto depende del día, el logueo o el nivel de rating.
- **Descuento**: efectivo solo en ramas de usuarios activos o logueados, condicionado por el precio o el número de páginas vistas.
- **Edad**: utilizada para discriminar dentro de no logueados con navegación intermedia; su efecto es marginal.

| Categoría técnica                      | Definición específica                                                                 |
|---------------------------------------|----------------------------------------------------------------------------------------|
| **Variable de Segmentación Raíz**     | Variable utilizada en el primer nivel del árbol para dividir el conjunto (nivel 0).   |
| **Condición Estructurante**           | Variable ubicada en el segundo nivel del árbol (nivel 1), estructura grandes ramas.   |
| **Condición Intermedia**              | Variable ubicada en niveles medios (nivel 2 o 3), refina subdivisiones parciales.     |
| **Criterio Final de Decisión**        | Variable que aparece en las hojas (nivel 4), determina valores predichos finales.     |
| **Variable estructural**              | Cualquier variable que participa en al menos una partición dentro del árbol.          |
| **No utilizada**                      | Variable que no participa en ninguna división del árbol de decisión.                  |



### Jerarquía y relaciones entre categorías

- **Variable estructural** es la categoría más amplia: agrupa todas las variables que participan en algún punto del árbol.
- Las subcategorías (**Variable de Segmentación Raíz**, **Condición Estructurante**, **Condición Intermedia**, **Criterio Final de Decisión**) son **mutuamente excluyentes** y se definen por el **nivel más superficial** en el cual aparece la variable:
  - Variable de Segmentación Raíz → nivel 0  
  - Condición Estructurante → nivel 1  
  - Condición Intermedia → niveles 2–3  
  - Criterio Final de Decisión → nivel 4
- Una variable no puede ocupar más de una subcategoría: se clasifica según su primer nivel de aparición.

### Síntesis comparativa de enfoques: regresiones lineales vs árbol de decisión

- **Las regresiones OLS** permiten estimar efectos marginales promedio y cuantificar la significación estadística de cada variable bajo un modelo aditivo. Son especialmente útiles para interpretar el peso de factores como `logueado`, `codigo_promocional`, `paginas_visitadas` o `domingo`, todos ellos con coeficientes significativos y estables.

- **El árbol de decisión**, en cambio, permite capturar relaciones no lineales, interacciones jerárquicas y puntos de quiebre. Variables como `paginas_visitadas`, `logueado`, `precio` y `rating_promedio`, que en la regresión no destacan por igual, emergen como criterios de división centrales en el árbol.

- La combinación de ambos enfoques revela:
  - Efectos lineales y globales (OLS)
  - Segmentaciones locales y umbrales decisivos (árbol)
  - Importancia contextual de variables que, según el modelo, pueden parecer irrelevantes si se observan de forma aislada

**Entonces**: modelar requiere abordar tanto relaciones promedio como estructuras condicionales. Usar solo una técnica puede ocultar patrones relevantes. La complementariedad entre métodos mejora tanto la capacidad explicativa como la robustez analítica.

---

### Qué hacer? ACCIONES basadas en el árbol de decisión y los modelos de regresión

---


Las decisiones óptimas deben alinearse con los patrones revelados en el árbol: no existen efectos universales de precio o descuento, sino reglas condicionadas por el comportamiento y estado del usuario. La regresión aporta señales de dirección general, pero es el árbol el que estructura las acciones. A continuación, se sistematizan las recomendaciones según los segmentos principales.

---

### 1. **Segmentar la estrategia en función de `paginas_visitadas`**

Esta variable aparece como raíz del árbol y organiza todo el flujo de decisión. Permite distinguir tres perfiles claramente diferenciados en términos de compromiso y respuesta:

#### a. **Usuarios con navegación baja (`paginas_visitadas ≤ 5.5`)**

- **Perfil**: primera visita, poca exploración, alto abandono si no hay impacto inmediato.
- **Acción**: mostrar beneficios de forma inmediata, en la primera pantalla. Usar testimonios, estrellas, beneficios visuales.
- **Evidencia**:
  - Si no están logueados y el rating es bajo, hay una hoja con predicción 6.00, pero con N = 1.
  - Si no están logueados y el rating es alto, un precio bajo (≤ 13.09) lleva a predicción 5.00, pero nuevamente N = 1.
  - Si están logueados, los días de semana y precios moderados (≤ 17.02) generan ventas de 3.50 (aunque N = 4).
- **Conclusión**: hay combinaciones que predicen mucho, pero no tienen respaldo muestral. Deben usarse con precaución. El precio solo ayuda si hay señales positivas (rating alto, logueo, día hábil).

#### b. **Usuarios con navegación intermedia (`5.5 < paginas_visitadas ≤ 9.5`)**

- **Perfil**: muestran cierto interés pero no alcanzan niveles intensos de exploración. Segmento más representativo en tamaño.
- **Acción**: diferenciar según si están logueados y según edad.
- **Evidencia**:
  - No logueados: los más jóvenes (≤ 37.33) responden un poco mejor (1.33 vs. 1.55).
  - Logueados con descuentos bajos (≤ 38.25): predicción 1.60 con N alto (1661).
  - Logueados con descuentos altos: sube a 3.60, pero con N = 5.
- **Conclusión**: el descuento tiene efecto sólo cuando hay login. Sin login, la segmentación por edad tiene efectos menores. Los descuentos agresivos deben reservarse para usuarios comprometidos.

#### c. **Usuarios con navegación alta (`paginas_visitadas > 9.5`)**

- **Perfil**: muy activos, muestran fuerte interés.
- **Acción**: aplicar precios moderados y mostrar descuentos sólo si hay evidencia de interés en horarios específicos.
- **Evidencia**:
  - Si `precio ≤ 19.28` y `descuento > 20.15`: predicción 4.50 (N = 2).
  - Si `precio > 19.28` y `hora > 1.5`: predicción 1.86 con N = 255.
- **Conclusión**: este es el segmento más sensible a combinaciones entre horario y precio. Las reglas son específicas y de aplicación precisa.

---

### 2. **No aplicar descuentos universales**

- **Árbol**: el `descuento` solo mejora ventas cuando se combina con navegación media-alta y login. 
- **Regresión**: coeficiente negativo (-0.0022), lo que indica que en promedio el efecto es negativo.
- **Acción**: restringir promociones fuertes a quienes ya hayan dado señales de interés (páginas vistas, login).

---

### 3. **Evitar confiar en precios bajos como estrategia general**

- **Árbol**: el `precio` sólo mejora predicciones en contextos acotados (por ejemplo, usuarios no logueados con rating alto).
- **Regresión**: el coeficiente de `precio` es estadísticamente irrelevante.
- **Acción**: utilizar precios bajos solo si otras señales acompañan. No asumir que bajar precios aumenta ventas.

---

### 4. **Aprovechar el login como condición estructurante**

- Aparece como segunda decisión más importante en el árbol (nivel 1).
- En la regresión tiene un coeficiente positivo y significativo (+0.1229).
- Estructura la forma en que operan otras variables: día, descuento, precio, edad.
- **Acción**: priorizar a usuarios logueados para acciones personalizadas, promociones y precios dinámicos.

---



# ![](./aux/recomendaciones.png)


### **Cruzar modelos: usar árbol y regresión estadística como complementos**

- **El árbol** detecta combinaciones jerárquicas que predicen mejor en segmentos específicos.
- **La regresión** captura efectos promedio que operan a nivel global.
- **Ejemplo**:
  - `edad` tiene un papel decisivo en el árbol bajo ciertas condiciones (no logueados), pero no es significativa en la regresión.
  - `clicks` no aparece en la regresión pero en el árbol define rutas con valores extremos (aunque con N bajo).
- **Acción**: combinar ambas fuentes para no sobreinterpretar reglas con baja representatividad, ni descartar efectos relevantes por estar diluidos en promedios.

---

# Categorical Decision Trees: cómo, cuándo y dónde? Por qué?



### Tipos de Árboles de Decisión: MSE, Entropía y Gini

Los árboles de decisión pueden optimizar distintos criterios según el tipo de problema:

---

#### 1. **Árbol de Regresión** (`criterion='squared_error'` o `MSE`)
- **Objetivo**: predecir una variable continua minimizando el **Error Cuadrático Medio (MSE)** en cada partición.
- **Funcionamiento**: en cada división, se busca el umbral que genere subconjuntos que minimicen la varianza de la variable dependiente.
- **Interpretación**: útil para capturar valores puntuales, con cortes más sensibles a valores extremos.
- **Ventaja**: alta precisión cuando se necesita modelar una variable numérica continua.

---

#### 2. **Árbol de Clasificación**

##### **(Entropía)** (`criterion='entropy'`)
- **Objetivo**: clasificar en categorías minimizando la **entropía de Shannon**, es decir, maximizando la pureza informativa de los nodos.
- **Funcionamiento**: mide la impureza como la cantidad de información (bits) necesaria para codificar una clase. Se elige el corte que más reduzca la incertidumbre.
- **Interpretación**: útil cuando se desea entender la estructura jerárquica de decisiones con alta precisión teórica.
- **Ventaja**: más sensible que Gini a diferencias pequeñas entre clases.

##### **(Gini)** (`criterion='gini'`)
- **Objetivo**: clasificar en categorías minimizando el **índice de Gini**, que mide la probabilidad de clasificación incorrecta.
- **Funcionamiento**: en cada corte se busca maximizar la pureza de los subconjuntos respecto a una clase dominante.
- **Interpretación**: favorece divisiones que agrupan de forma más homogénea.
- **Ventaja**: computacionalmente más eficiente que la entropía y da resultados similares en muchos casos.



#### Primera aproximación a diferencias clave

| Criterio   | Tipo de Variable | Métrica           | Sensibilidad a clases | Enfoque principal        |
|------------|------------------|--------------------|------------------------|--------------------------|
| MSE        | Continua          | Varianza           | No aplica              | Precisión numérica       |
| Entropía   | Categórica        | Información (Information Gain)       | Alta                   | Pureza informativa       |
| Gini       | Categórica        | Impureza estadística| Moderada              | Dominancia de clase      |

---


### Comparación técnica de criterios de partición en árboles de decisión

| Criterio     | Tipo de Variable  | Fórmula matemática                                                                                     | Naturaleza        | Sensibilidad a clases | Interpretación técnica                                         |
|--------------|-------------------|----------------------------------------------------------------------------------------------------------|--------------------|------------------------|----------------------------------------------------------------|
| **MSE**      | Continua           | $ \text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \bar{y})^2 $                                             | Error cuadrático   | No aplica              | Mide la varianza dentro de los nodos. Menor MSE implica mayor homogeneidad numérica. Ideal para regresión. |
| **Entropía** | Categórica         | $ H(S) = -\sum_{i=1}^{c} p_i \log_2(p_i) $                                                             | Logarítmica        | Alta                   | Mide la incertidumbre del sistema. Penaliza más la mezcla de clases. Tiende a árboles más profundos.        |
| **Gini**     | Categórica         | $ G(S) = 1 - \sum_{i=1}^{c} p_i^2 $                                                                    | Cuadrática         | Moderada               | Mide la impureza. Es computacionalmente más simple que la entropía. Favorece clases dominantes.            |


#### Donde:

- $ p_i $: proporción de elementos de la clase $ i $ en el nodo.
- $ c $: número total de clases posibles.
- $ y_i $: valor observado.
- $ \bar{y} $: promedio de valores del nodo.

---

#### Consideraciones prácticas

- **MSE**:
  - Se usa exclusivamente en árboles de **regresión** con VARIABLE CONTINUA.
  - Busca reducir la varianza intra-nodo.
  - Cortes tienden a sobreajustarse si hay valores extremos.

---

- Los trees DE CLASIFICACIÓN sea con criterios de **entropía o gini**, se usan con variables DISCRETAS (0-1, o ALTO, MEDIO, BAJO; etc.):
    - **Entropía**:
        - Se usa en **clasificación**, priorizando máxima ganancia informativa.
        - Es más sensible a distribuciones desbalanceadas.
        - Tiene mayor coste computacional por el logaritmo.

    - **Gini**:
        - Alternativa a la entropía, más eficiente.
        - Suaviza el efecto de clases raras.
        - Prefiere divisiones con una clase claramente mayoritaria.

---

---


### ¿Qué hace un Árbol de Clasificación?

- Es un algoritmo que predice categorías (valores discretos) mediante divisiones sucesivas del espacio de datos.
- En cada división (split), se elige la variable y el umbral que maximiza la "pureza" de los grupos.
- Para eso se usan métricas como `entropía` o `índice Gini`, que indican qué tan mezcladas están las clases.
- El resultado es una estructura de árbol con reglas condicionales que separan mejor las clases, rama por rama.

---

### ¿Cómo se mide la pureza de un nodo?

Se utiliza una métrica de impureza. La más común es la `entropía`, definida como:

$$
\text{Entropía o H(S)} = -\sum_{i} p_i \cdot \log_2(p_i)
$$

Donde $ p_i $ es la proporción de elementos de la clase $ i $ en el nodo. $H(S)$ se lee en unidades "bits". Cero bits: evento seguro, muchos bits: evento raro. A más bits, más inesperado en el evento. 

- Si un nodo tiene sólo una clase → entropía = 0 → nodo completamente puro.
- Si hay mezcla de clases → entropía > 0 → nodo impuro.
- El árbol busca dividir los datos en nodos con la menor entropía total posible.

---

### ¿Qué representa cada componente?

- $ p_i $: es la **proporción** (o probabilidad) de la clase `i` en el nodo.
- $ \log_2(p_i) $: mide cuánta **información** aporta un evento si ocurre la clase `i`. Es negativa (porque $ p_i < 1 $) y su valor absoluto es mayor cuando la clase es más improbable.
- El producto $ p_i \cdot \log_2(p_i) $ mide la "información promedio esperada" para una clase. Al sumar todos, se obtiene la incertidumbre total del nodo.

> El logaritmo aparece porque en teoría de la información, la cantidad de información de un evento con probabilidad $p$ es $-\log_2(p)$. Cuanto más improbable es el evento, más información genera al ocurrir.

---

##### ¿Por qué se multiplica por $ \log_2(p_i) $?

$\log_2()$ es una función que transforma la probabilidad en una medida de información, indicando formalmente cuán "inesperado" o "raro" es que ocurra un evento de probabilidad $p$:

- Si $p_A = 1$, el evento es seguro → $\log_2(1) = 0$ → no aporta información. Es decir, no es raro que pase "A".
- Si $p_B = 0.5$, el evento es incierto → $\log_2(0.5) = -1$ → aporta 1 bit de información. Es decir, "B" es raro la mitad de las veces.
- Si $p_B = 0.25$, el evento es más raro → $\log_2(0.25) = -2$ → aporta 2 bits. Más inesperado el evento, más bits.
- Si $p_B = 0.1$, el evento es muy raro → $\log_2(0.1) \approx -3.32$ → aporta más de 3 bits. Es decir, es muy raro que pase "B".

$$ H(S) = -\sum_{i=1}^{c} p_i \log_2(p_i) $$         

    Nota: por eso el signo negativo al inicio de la fórmula, ya que la transformación log_2 devuelve valores negativos. La interpretación es absoluta. 

##### ¿Por qué se multiplica por $ p_i $?

Porque buscamos una media ponderada: el valor esperado de la sorpresa que representa cada clase, en función de su probabilidad. Nos interesa saber cuánta información "trae" en promedio un caso aleatorio de ese nodo.

<center> <h2><b>La entropía mide cuán mezcladas están las clases en un conjunto de datos</b></h2></center>

---

### Ejemplo con tres clases (`A`, `B`, `C`)

| precio | clase |
|--------|-------|
| 10     | A     |
| 15     | A     |
| 20     | B     |
| 25     | B     |
| 30     | C     |

Queremos encontrar el mejor punto de corte en la variable `precio`.

---

#### Corte: `precio ≤ 17.5`

- Grupo 1: [10, 15] → clases: A, A  
  - $ p_A = 1.0 $  
  - Entropía = $ -1 \cdot \log_2(1) = 0.0 $

- Grupo 2: [20, 25, 30] → clases: B, B, C  
  - $ p_B = 2/3,\ p_C = 1/3 $  
  - Entropía = $ -\left( \frac{2}{3} \log_2 \frac{2}{3} + \frac{1}{3} \log_2 \frac{1}{3} \right) ≈ 0.918 $

**Entropía total (ponderada):**

$$
\text{Entropía total} = \frac{2}{5} \cdot 0 + \frac{3}{5} \cdot 0.918 = 0.551
$$

> Este valor se compara con los de otros cortes para seleccionar el óptimo.

---

### ¿Qué pasa en casos extremos?

- Si el nodo tiene 100% de clase A:  
  $$
  \text{Entropía} = -1 \cdot \log_2(1) = 0
  $$

- Si las clases están perfectamente balanceadas, por ejemplo 50% A y 50% B:  
  $$
  \text{Entropía} = -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 1
  $$

- Si hay 80% A y 20% B:  
  $$
  \text{Entropía} = -0.8 \log_2(0.8) - 0.2 \log_2(0.2) ≈ 0.7219
  $$

> A medida que las clases se desbalancean, la entropía baja: el nodo es más predecible.

---

### Conclusiones del ejemplo con clases `A`, `B`, `C`

1. **La entropía sirve para evaluar cortes, no para predecir clases.**
   - En este ejemplo, el objetivo no es asignar una clase aún, sino decidir cómo dividir los datos para que los grupos resultantes sean lo más homogéneos posible.

2. **El corte `precio ≤ 17.5` separa perfectamente a los casos de clase `A`.**
   - El grupo 1 (precio ≤ 17.5) contiene sólo casos de clase `A`, por lo tanto su entropía es `0.0`. Es un grupo completamente puro.
   - El grupo 2 contiene una mezcla de `B` y `C`, lo que genera una entropía positiva (≈ 0.918).

3. **La entropía total del corte (0.551) refleja la mezcla residual en los grupos.**
   - Es un promedio ponderado de las entropías de los grupos según su tamaño relativo.
   - Esta entropía total representa la calidad de la división: cuanto más baja, mejor.
   - *El corte `precio ≤ 17.5` tiene entropía de $0.551$ bits. 

4. **Comparamos entre todos los cortes posibles para seleccionar el óptimo.**
   - En este ejemplo, si los demás cortes (como 22.5 o 27.5) tuvieran entropías totales mayores que 0.551, entonces `precio ≤ 17.5` sería el mejor split.

5. **La entropía indica cuán predictivo será un nodo hoja, pero no es la predicción.**
   - En el grupo 1, donde la entropía es cero, sabemos con certeza que cualquier nuevo dato que caiga allí será de clase `A`.
   - En el grupo 2, aunque no es puro, el modelo aún puede asignar una clase mayoritaria (`B`) como predicción, pero con cierta incertidumbre.

6. **Casos extremos ilustran el comportamiento de la entropía:**
   - La entropía mínima es siempre `0` y ocurre cuando todos los casos en el nodo pertenecen a una sola clase (máxima pureza).
   - La entropía máxima depende del número total de clases `k`, y se alcanza cuando todas las clases están presentes en proporciones iguales:
     
     $$
     \text{Entropía}_\text{máx} = \log_2(k)
     $$

      Por ejemplo:
      - Con 2 clases: $\log_2(2) = 1$
      - Con 3 clases: $\log_2(3) ≈ 1.585$
      - Con 4 clases: $\log_2(4) = 2$
      - Con 10 clases: $\log_2(10) ≈ 3.322$

   - Valores intermedios reflejan distintos grados de mezcla: a mayor desbalance entre clases, menor entropía → mayor certeza predictiva.

---



### Aplicación real: clasificación de `ventas_cat`

Supongamos que discretizamos las ventas reales en tres clases:

- `nada` (ventas = 0)
- `una` (ventas = 1)
- `mas_de_una` (ventas > 1)

Usamos la variable `precio` para intentar dividir los datos.

| precio | ventas_cat  |
|--------|-------------|
| 18     | nada        |
| 21     | mas_de_una  |
| 24     | nada        |
| 27     | una         |
| 30     | mas_de_una  |

---

#### Corte: `precio ≤ 25.5`

- Grupo 1: [18, 21, 24]  
  - Clases: nada, mas_de_una, nada  
  - $ p_{nada} = 2/3,\ p_{mas} = 1/3 $  
  - Entropía ≈ 0.918

- Grupo 2: [27, 30]  
  - Clases: una, mas_de_una  
  - $ p_{una} = 0.5,\ p_{mas} = 0.5 $  
  - Entropía = 1.0

**Entropía total:**

$$
\text{Entropía total} = \frac{3}{5} \cdot 0.918 + \frac{2}{5} \cdot 1.0 = 0.951
$$

> Si otro corte genera una entropía total menor, será preferido por el árbol.

---

### Conclusiones del ejemplo aplicado a `ventas_cat`

1. **El árbol de clasificación evalúa cortes para reducir la mezcla entre clases.**
   - En este caso, se busca dividir la variable `precio` para lograr nodos más homogéneos respecto de `ventas_cat` (nada, una, más_de_una).

2. **El corte `precio ≤ 25.5` genera grupos con entropías moderadamente altas.**
   - El Grupo 1 tiene dos clases: `nada` (66%) y `mas_de_una` (33%) → Entropía ≈ 0.918
   - El Grupo 2 tiene `una` (50%) y `mas_de_una` (50%) → Entropía = 1.0
   - Aunque el Grupo 1 tiene una mayoría clara, el Grupo 2 está perfectamente mezclado entre dos clases, lo que incrementa la incertidumbre.
   - **La entropía total ponderada del corte (≈ 0.951) representa la impureza residual.**
   - Resume cuán "mixtas" quedaron las clases después del split. Es el criterio que el árbol usa para decidir si ese punto de corte es bueno.

3. **El árbol buscará un corte con menor entropía total si lo hay.**
   - Si otro umbral de `precio` produce una división donde uno o ambos grupos son más homogéneos (con menos entropía), se preferirá ese.

4. **El corte no predice aún una clase, pero prepara el camino para decidirla.**

Cuando se aplica un corte (por ejemplo, `precio ≤ 25.5`), el árbol no está prediciendo directamente una clase. Lo que hace en ese momento es dividir el conjunto de datos en dos grupos distintos, según los valores de esa variable.

Cada grupo resultante contiene casos que pertenecen a distintas clases, y el objetivo es que uno de esos grupos sea más "puro" (es decir, que tenga una clase mayoritaria clara).

Luego del corte:

- En cada grupo, el árbol evalúa la distribución de clases.
- Si una clase aparece con más frecuencia que las demás, se considera la **clase mayoritaria** y se utiliza como predicción en ese nodo hoja.
- El **corte define los subconjuntos** en los que se tomará una decisión.
- La **predicción de clase** se hace una vez que ya no se puede (o no conviene) dividir más.

5. **Este proceso se repite recursivamente en cada rama del árbol.**
   - Luego del primer split (`precio ≤ 25.5`), cada grupo puede ser subdividido nuevamente usando otra variable que reduzca la entropía local.
   - Así se construye la jerarquía de reglas que forma el árbol completo.

---

### Comparación con árboles de regresión

- En regresión se mide el **error cuadrático medio** (MSE), que representa la dispersión respecto a un valor promedio:

$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \bar{y})^2
$$

- En clasificación, se mide la **impureza** del nodo, por ejemplo con `entropía`, que mide la mezcla de clases.

- Ambos enfoques buscan el mismo objetivo: nodos lo más homogéneos posible según la variable objetivo.

---

---

### ¿Cómo se predice una clase en un Árbol de Clasificación?

Una vez que el árbol ha dividido los datos mediante cortes sucesivos, cada ruta termina en un **nodo hoja**. Allí es donde se asigna finalmente una **predicción de clase**. Este proceso se basa en los siguientes pasos:

Cada vez que el árbol evalúa una variable (por ejemplo, precio ≤ 25.5), divide el conjunto de datos en dos ramas: una a la izquierda (cumple la condición) y otra a la derecha (no la cumple). Al seguir una decisión tras otra —cada una con su condición— se va recorriendo una ruta única.

Por ejemplo:

- Si `paginas_visitadas ≤ 5.5`  
- Y `logueado == 0`  
- Y `rating_promedio ≤ 3.2`  
→ entonces se predice: clase `'nada'`

---

### 1. Cada caso del dataset atraviesa el árbol

En cada nivel del árbol, se aplica una condición (por ejemplo: `precio ≤ 25.5`).  
Si el caso cumple la condición, va por la **rama izquierda**. Si no, va por la **rama derecha**.  
Esto se repite con otras variables y umbrales, hasta que el caso cae en un nodo final.

---

### 2. El nodo hoja (leaf)

Todos los casos que siguen la misma secuencia de decisiones terminan en un mismo nodo hoja.

Estos puntos de corte fueron seleccionados por ser los que minimizan entropía.

---

### 3. En la hoja, se cuenta cuántos casos hay de cada clase

Una vez que el árbol ha clasificado los datos del entrenamiento, cada nodo hoja contiene un subconjunto de observaciones. Se contabilizan cuántas observaciones corresponden a cada clase:

| Clase         | Frecuencia |
|---------------|------------|
| `nada`        | 3          |
| `una`         | 1          |
| `mas_de_una`  | 6          |

Estas frecuencias se obtienen contando cuántos casos del entrenamiento que llegaron a ese nodo pertenecen a cada clase.

---

### 4. El árbol predice la clase más frecuente del nodo hoja

- En el ejemplo anterior, la clase `mas_de_una` es la más frecuente → esa será la predicción para ese nodo hoja.
- Si luego un nuevo dato (no visto) recorre esa misma ruta, el modelo le asignará la misma clase (`mas_de_una`).

---

### 5. ¿Qué pasa si el nodo está muy mezclado?

Si el nodo hoja contiene varias clases en proporciones similares (por ejemplo: 4 casos de `una`, 3 de `nada`, 3 de `mas_de_una`):

- La entropía será alta, indicando mayor incertidumbre.
- El árbol de todas formas asignará la clase más frecuente (aunque la diferencia sea mínima).
- Esa predicción será menos confiable, ya que la clase mayoritaria tiene poca ventaja frente a las otras.

---

### 6. La entropía no predice, pero anticipa la incertidumbre

La entropía no da una clase como salida, pero indica:

- Cuánto podemos confiar en la predicción de ese nodo (o combinaciones de cortes, que llevan a ese nodo).
- Si la entropía es alta, hay mezcla de clases → la predicción tendrá más margen de error.
- Si es baja (cercana a 0), la predicción será confiable.



# Ejemplo de tree de clasificación con entropía

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mis_funciones.view_dectree import view_dectree
from mis_funciones.view_detail_text_cattree import view_text_cattree_detailed, view_text_cattree_pruned


# Cargar datos
df = pd.read_csv("dataset_suscripciones.csv")

# Clasificación de variable target

# Separamos la variable ventas_cat en tres valores: NADA, UNA y MÁS DE UNA

df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

# Usamos -1 como límite inferior para asegurar que 0 quede en la categoría 'nada'
# Esto es porque el método pd.cut() excluye el límite superior, por lo que 0 quedaría sin categoría:  pd.cut cierra el límite izquierdo y abre el derecho: (a, b]
# Si usáramos [0,1,2] los valores = 0 quedarían excluidos del primer bin
# Con [-1,0,1] capturamos:
#   - 'nada': ventas = 0
#   - 'una': ventas = 1  
#   - 'mas_de_una': ventas > 1


X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()

y = df['ventas_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del árbol con entropía
clf = DecisionTreeClassifier(max_depth=4, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

# visualización del Árbol en texto plano, sin detalles 
tree_text = export_text(clf, feature_names=list(X.columns))
print("\n=== ÁRBOL DE CLASIFICACIÓN (salida básica default) ===\n")
print(tree_text)

# Visualización simple del árbol
plt.figure(figsize=(12, 8)) 
tree.plot_tree(clf, filled=True)
plt.show()


# --- Visualización avanzada de árbol, función propia, ver mis_funciones.py | detallada y editable---
view_dectree(
    tree_model=clf,
    features=list(X.columns),
    nombre_base='arbol_clasificacion_ventas',
    horizontal_spacing=1.0,
    vertical_spacing=1.2,
    font_size=16,
    flecha_grosor_factor=2,
    min_leaf_pct=0.01
)

print("\n=== Árbol categórico con métricas ===\n")
print(view_text_cattree_detailed(clf, X.columns))

print("\n=== Árbol categórico con poda visual (hojas <0.5% ocultas) ===\n")
print(view_text_cattree_pruned(clf, X.columns, min_pct=0.5))

# Evaluación -  indicadores básicos para el modelo de clasificación
print("\n=== EVALUACIÓN DEL MODELO DE CLASIFICACIÓN ===\n")
y_pred = clf.predict(X_test)
y_test_str = y_test.astype(str)
y_pred_str = pd.Series(y_pred).astype(str)
acc = accuracy_score(y_test_str, y_pred_str)
prec = precision_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
rec = recall_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
f1 = f1_score(y_test_str, y_pred_str, average='weighted', zero_division=0)

print(f"Accuracy: {acc:.4f}")
print(f"Precision (weighted): {prec:.4f}")
print(f"Recall (weighted): {rec:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

### Métricas básicas para evaluar árboles de decisión categóricos

Cuando el árbol de decisión clasifica una variable categórica (como `ventas_cat`), las métricas cambian respecto al caso de regresión. En lugar de calcular cuánto se aleja una predicción de un valor continuo (como en MSE), se evalúa si la clase predicha coincide o no con la real, y cómo se distribuyen los errores entre las distintas clases. Las principales métricas son:

---

#### `Accuracy`
- **Qué mide:** proporción de predicciones correctas sobre el total.
- **Fórmula:** `accuracy = (TP + TN) / Total` (solo válida en binaria; en multiclase se generaliza como aciertos totales / total casos).
- **Interpretación:** indica qué porcentaje del total fue correctamente clasificado.
- **Lectura típica:** `Accuracy (exactitud): 0.4070` → el 40.70% de las observaciones fueron correctamente clasificadas.

- **`TP (True Positives)`**: cantidad de casos que el modelo predijo como positivos y que realmente eran positivos.  
  Ejemplo: el modelo predice "compra" y efectivamente el usuario compró.

- **`TN (True Negatives)`**: cantidad de casos que el modelo predijo como negativos y que realmente eran negativos.  
  Ejemplo: el modelo predice "no compra" y efectivamente el usuario no compró.




Estas categorías son claras en un problema binario (dos clases). En problemas multiclase, se generalizan evaluando clase por clase como si fuera binaria (uno contra el resto), y luego se agregan los resultados.

---

#### `Precision` (ponderada)
- **Qué mide:** de todos los casos que el modelo etiquetó como pertenecientes a una clase, cuántos realmente lo eran.
- **Fórmula por clase:** `precision = TP / (TP + FP)`

- **`FP (False Positives)`**: cantidad de casos que el modelo predijo como positivos pero que realmente eran negativos.  
  Ejemplo: el modelo predice "compra" pero el usuario no compró.
- **En promedio ponderado:** cada clase contribuye en proporción a su tamaño.
- **Interpretación:** muestra cuántas predicciones correctas hizo el modelo entre todos los positivos que predijo. Alta precisión = pocas falsas alarmas.
- **Lectura típica:** `Precision (ponderada): 0.2883` → muchas predicciones no coincidieron con su clase real; baja fiabilidad por categoría.

---

#### `Recall` (sensibilidad, ponderado)
- **Qué mide:** de todos los casos reales de una clase, cuántos fueron correctamente predichos.
- **Fórmula por clase:** `recall = TP / (TP + FN)`
- **`FN (False Negatives)`**: cantidad de casos que el modelo predijo como negativos pero que realmente eran positivos.  
  Ejemplo: el modelo predice "no compra" pero el usuario compró.
- **En promedio ponderado:** pondera por soporte (cantidad de verdaderos casos).
- **Interpretación:** indica cuántos positivos reales fueron detectados. Bajo recall = el modelo "no ve" muchas ocurrencias reales.
- **Lectura típica:** `Recall (ponderado): 0.4070` → el modelo reconoce el 40.70% de los verdaderos casos de cada clase.

#### `F1 Score` (ponderado)
- **Qué mide:** equilibrio entre precisión y recall. Castiga tanto los falsos positivos como los falsos negativos.
- **Fórmula:** `F1 = 2 * (precision * recall) / (precision + recall)`
- **Interpretación:** útil cuando queremos asegurar que ni la precisión ni el recall son bajos. 
- **Lectura típica:** `F1 Score (ponderado): 0.3244` → el modelo no logra balancear bien aciertos vs. errores.

---

---

### Diferencias con Regresion Tree
- En regresión se evalúan desviaciones (`MSE`, `RMSE`, etc.) de una variable continua. No hay clases ni categorías.
- En clasificación, se mide qué tan bien se distingue entre clases discretas, sin importar la magnitud del error numérico.
- Las métricas de clasificación permiten identificar sesgos estructurales (por ejemplo, no detectar nunca una clase).

---

### Comparación estructural entre Árbol de Regresión (MSE) y Árbol Categórico (Entropía)


<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tabla de Comparación de Árboles</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.6;
            color: #000000;
            background-color: #f8f9fa;
            margin: 0;
            padding: 20px;
            display: flex;
            justify-content: center;
        }
        .table-container {
            width: 900px;
            overflow-x: auto;
            background-color: #ffffff;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            padding: 15px;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            font-size: 0.9em;
        }
        caption {
            font-size: 1.5em;
            font-weight: bold;
            margin-bottom: 15px;
            color: #000000;
            text-align: left;
        }
        th, td {
            border: 1px solid #dee2e6;
            padding: 12px 15px;
            text-align: left;
            vertical-align: top;
        }
        thead th {
            background-color: #007bff;
            color: #000000;
            font-weight: bold;
            position: sticky;
            top: 0;
            z-index: 1;
        }
        tbody tr:nth-child(even) {
            background-color: #f2f2f2;
        }
        tbody tr:hover {
            background-color: #e9ecef;
            cursor: default;
        }
        td:first-child {
            font-weight: 500;
            color: #000000;
        }
        td {
            line-height: 1.5;
            color: #000000;
        }
        td code {
            background-color: #e8e8e8;
            padding: 2px 4px;
            border-radius: 3px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            color: #000000;
        }
        /* Nuevo estilo para separar las columnas */
        th:nth-child(4), td:nth-child(4) {
            border-left: 3px solid #007bff;
        }
    </style>
</head>
<body>

<div class="table-container">
    <table>
        <caption>Comparativa de Modelos de Árboles de Decisión</caption>
        <thead>
            <tr>
                <th>Variable</th>
                <th>Árbol de Regresión (MSE)</th>
                <th>Interacción estructural en árbol (MSE)</th>
                <th>Árbol Categórico (Entropía)</th>
                <th>Interacción estructural en árbol (Entropía)</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Clicks</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 0, rating ≤ 2.53</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 0, rating ≤ 3.84</td>
            </tr>
            <tr>
                <td>Tiempo_Browsing</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 0, rating ≤ 2.53</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 0, rating ≤ 3.84</td>
            </tr>
            <tr>
                <td>Paginas_Visitadas</td>
                <td>Variable de Segmentación Raíz (nivel 0)</td>
                <td>estructura todas las ramas</td>
                <td>Variable de Segmentación Raíz (nivel 0)</td>
                <td>estructura todas las ramas</td>
            </tr>
            <tr>
                <td>Domingo</td>
                <td>Condición Intermedia (nivel 2)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 1</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 1</td>
            </tr>
            <tr>
                <td>Dia_Semana</td>
                <td>Condición Intermedia (nivel 2)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 1</td>
                <td>-</td>
                <td>-</td>
            </tr>
            <tr>
                <td>Hora</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas > 9.5, precio > 19.28</td>
                <td>-</td>
                <td>-</td>
            </tr>
            <tr>
                <td>Logueado</td>
                <td>Condición Estructurante (nivel 1)</td>
                <td>cuando paginas_visitadas ≤ 5.5 y también en ramas > 5.5</td>
                <td>Condición Intermedia (nivel 2–3)</td>
                <td>en ambas ramas, diferencia uso de descuento y rating</td>
            </tr>
            <tr>
                <td>Codigo_Promocional</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas > 5.5, logueado == 1</td>
                <td>Condición Intermedia (nivel 2)</td>
                <td>cuando paginas_visitadas ≤ 5.5</td>
            </tr>
            <tr>
                <td>Precio</td>
                <td>Condición Intermedia (niveles 2–3)</td>
                <td>cuando rating > 2.53 o logueado == 1</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>en ramas anónimas con rating alto o fin de semana</td>
            </tr>
            <tr>
                <td>Descuento</td>
                <td>Condición Intermedia (niveles 2–3)</td>
                <td>cuando paginas_visitadas > 5.5, logueado == 1</td>
                <td>Condición Intermedia (nivel 3–4)</td>
                <td>en ramas de páginas altas, afecta especialmente edad > 25</td>
            </tr>
            <tr>
                <td>Rating</td>
                <td>Condición Estructurante (nivel 1)</td>
                <td>cuando paginas_visitadas ≤ 5.5, logueado == 0</td>
                <td>Condición Intermedia (nivel 2–3)</td>
                <td>en ambas ramas, activa criterio sobre precio o edad</td>
            </tr>
            <tr>
                <td>Edad</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas ≤ 9.5, logueado == 0</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas > 9.5</td>
            </tr>
            <tr>
                <td>Codigo_Promocional</td>
                <td>Criterio Final de Decisión (nivel 4)</td>
                <td>cuando paginas_visitadas > 5.5, logueado == 1</td>
                <td>Condición Intermedia (nivel 2)</td>
                <td>diferencia ramas entre usuarios anónimos o no, fin de semana</td>
            </tr>
            <tr>
                <td>Es_Domingo</td>
                <td>-</td>
                <td>-</td>
                <td>Condición Intermedia (nivel 3)</td>
                <td>bajo ramas con código promocional</td>
            </tr>
        </tbody>
    </table>
</div>

</body>
</html>


### Cómo comparar y qué implican las diferencias entre un Árbol de Regresión (MSE) y un Árbol Categórico (Entropía)


La tabla permite analizar **qué variables usa cada árbol, en qué niveles de decisión**, y **en qué condiciones contextuales**. La comparación es útil porque los modelos responden a objetivos distintos:

| Criterio de división | Árbol de Regresión           | Árbol Categórico              |
|----------------------|------------------------------|-------------------------------|
| Qué optimiza         | Reducción del **error cuadrático medio** MSE | Reducción de la **entropía de clase**  (mejora pureza)     |
| Qué busca            | Mejorar la **predicción del valor continuo** (CUÁNTAS ventas) | Mejorar la **clasificación correcta** (predecir categorías de venta) -> [0, 1, +1] |
| Por qué varía la estructura | Algunas variables explican mejor la **magnitud**, pero no discriminan bien clases | Algunas variables separan bien clases, aunque no mejoran la predicción continua |
| Ejemplo | `"hora" explica cuánto más se vende, cuando se vende`  | `"hora" no diferencia, no aparece en el árbol (su variación entra dentro de +1 en categorical)` |

#### Claves para comparar ramas y condiciones

- **Variables comunes en ambos árboles**: indican robustez —aportan tanto para clasificar como para predecir valores.
- **Variables exclusivas de un árbol**: indican especificidad —responden a un objetivo y no al otro.
- **Nivel de profundidad**: cuanto más arriba aparece una variable (menor nivel), más estructural es en ese árbol.
- **Contexto de uso**: se deben observar las condiciones bajo las cuales una variable entra en juego.

---

---

### Conclusión práctica

Si pueden, usen ambas aproximaciones. :) 


---
## Variables diferenciadas y recomendaciones de acción  
_Comparación entre Árbol de Regresión (MSE) y Árbol Categórico (Entropía)_

---

### 1. `Codigo_Promocional`

**Análisis comparado:**
- Aparece en **ambos modelos**, pero con **jerarquía distinta**:
  - **Regresión**: es un **criterio final**, de peso marginal.
  - **Clasificación**: es una **condición intermedia**, usada para segmentar grupos de usuarios.
- Esto sugiere que **su efecto es moderado para predecir el total**, pero **relevante para diferenciar clases**.

**Qué recomendar:**
- Activar `codigo_promocional` **solo para usuarios con navegación baja** (≤ 5.5 páginas).
- **Mostrar códigos de descuento visibles desde el inicio**, ya que el árbol categórico los identifica como impulsores de compra múltiple en ese segmento.

---

### 2. `Es_Domingo`

**Análisis comparado:**
- Aparece **solo en el árbol de clasificación**.
- No incide sobre el volumen de ventas total, pero **segmenta bien ciertos patrones de comportamiento** (p. ej. usuarios más proclives a comprar más de un ítem).

**Qué recomendar:**
- Usar `es_domingo` como **disparador simplificado de promociones**.
- En días domingo, **reducir personalización** y aplicar **campañas planas**, ya que otros factores pierden peso predictivo.

---

### 3. `Rating`, `Descuento` y `Tiempo_Browsing`

**Análisis comparado:**
- Están **presentes en ambos árboles**, pero con **niveles y funciones diferentes**.
  - `Rating` y `Descuento` tienen **efectos cruzados** sobre valor y clases.
  - `Tiempo_Browsing` aparece como **criterio final solo en clasificación**.
- Esto indica que su influencia **varía según el objetivo** (valor continuo vs. clase).

**Qué recomendar:**
- **Rating alto (> 3.84)**: aplicar **descuentos automáticos**, incluso sin login o historial profundo. El modelo muestra que sustituye otras señales.
- **Tiempo de navegación alto** con bajo rating: **reforzar estímulos** (ej. ventanas emergentes, CTA) aunque otros indicadores sean débiles.

---

### 4. `Edad`

**Análisis comparado:**
- Presente en ambos modelos, pero en diferentes contextos:
  - En regresión, como **criterio final puntual**.
  - En clasificación, **solo en usuarios con navegación alta** (> 9.5 páginas).
- Su impacto se restringe a **casos con fuerte involucramiento**.

**Qué recomendar:**
- En jóvenes con navegación alta y buen rating, usar **promociones cruzadas** o **bundles** para empujar la compra múltiple.

---

### 5. `Dia_Semana` y `Hora`

**Análisis comparado:**
- Aparecen **solo en el árbol de regresión**.
- Señalan **momentos del día o semana con mayor volumen de ventas**, pero **no aportan segmentación conductual clara**.

**Qué recomendar:**
- Usar estas variables para **planificar campañas horarias o semanales**, no para personalizar contenidos.
- Ejemplo: más ventas los viernes a la tarde → reforzar campañas de remarketing en ese slot.

---

---

# Overfitting


### ¿Qué es el overfitting en árboles de clasificación categórica?

---

## Explicación paso a paso

- **Overfitting** significa que el modelo "aprendió demasiado" los datos de entrenamiento.
- No aprendió los patrones generales, sino **detalles específicos** que no se repiten en datos nuevos.
- En vez de generalizar, **memoriza**: si ve el mismo dato, responde bien; si cambia un poco, falla.

### Ejemplo simple
Un árbol que clasifica usuarios según edad y clicks, y crea reglas como:
- "Si edad = 36 y clicks = 4, entonces clase = 'una'"
Eso funciona si alguien tiene justo esa combinación, pero no para casos nuevos.



- Accuracy: 0.4070
- Precision (weighted): 0.2883
- Recall (weighted): 0.4070
- F1 Score (weighted): 0.3244

---

## ¿Cómo se manifiesta en árboles categóricos?

- **Profundidad excesiva**: muchas divisiones que capturan ruido.
- **Hojas muy pequeñas**: el modelo llega a reglas con muy pocos casos (1 o 2).
- **Clase minoritaria sobreajustada**: inventa reglas raras para detectar los pocos casos de una clase poco frecuente.

---

## Señales de overfitting

- `Accuracy` en test más `baja` que en entrenamiento.
- `F1 score` `alto` sólo en clases frecuentes.
- `Recall` muy `bajo` en clases pequeñas.
- Árbol muy complejo con muchas ramas que no aportan mejora clara.
- `n BAJO GENERALIZADO en clases leaf (o hoja, o final del árbol)`


---





In [ ]:

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")

# Crear variable categórica target
df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

# Variables predictoras
X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo clasificación profundidad  = 4 --> a esta profundidad la definimos discresional y arbitrariamente. 
# Elegimos 4 porque en el regresion tree tenía sentido esa profundidad. Probemos acá a ver si da bien. 
clf = DecisionTreeClassifier(max_depth=4, criterion='entropy', random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


print("\n=== Evaluación del Modelo de Clasificación (profundidad 4) ===")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred, average='weighted', zero_division=0):.4f}")

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, zero_division=0))

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

print("\n--- Support por clase (n de cada categoría en test set) ---")
print(y_test.value_counts())

### Evaluación del Modelo de Clasificación (Decision Tree - Profundidad 4)
#### Métricas generales

| Métrica     | Valor   |
|-------------|---------|
| Accuracy    | 0.4070  |
| Precision   | 0.2883  |
| Recall      | 0.4070  |
| F1 Score    | 0.3244  |

- **Accuracy** indica que el modelo acierta en la clasificación (predicción) del 40.7% de los casos (todo el sample).
- **Precision baja**: muchas predicciones son incorrectas, especialmente para clases minoritarias (dentro de la clase relativa, no todo el sample).
- **Recall moderado**: el modelo identifica correctamente el 40.7% de los casos positivos reales (contrafactual real vs predicho - aciertos y e y_hat).
- **F1 Score** intermedio: refleja un equilibrio pobre entre precision y recall (es el promedio armónico $2*(recall*precision)/(recall+precision)$).


### Exploremos en detalle:  Diferencia entre Precision y Recall 

Ambas métricas evalúan el desempeño del modelo **por clase específica**, pero desde ángulos distintos: uno enfocado en lo que el modelo predice (precision), y otro en lo que realmente ocurre (recall).

---

### 1. **Precision**

- **Qué mide:** qué tan precisas son las predicciones de una clase.
- **Fórmula:**  
  $$
  \text{Precision} = \frac{TP}{TP + FP}
  $$

- **Interpretación:**  
  De todos los casos que el modelo **predijo como positivos** (por ejemplo: predijo "compra"), ¿cuántos **realmente lo eran**?

---

### 2. **Recall**

- **Qué mide:** qué tan bien el modelo encuentra los casos reales de una clase.
- **Fórmula:**  
  $$
  \text{Recall} = \frac{TP}{TP + FN}
  $$

- **Interpretación:**  
  De todos los casos que **realmente eran positivos** (por ejemplo: usuarios que compraron), ¿cuántos **fueron detectados** por el modelo?

---

### 3. **Ejemplito claro**

Supongamos que estamos analizando la clase `compra`:


| Caso | Clase real | Predicción del modelo |
|------|------------|------------------------|
| 1    | compra     | compra                 |
| 2    | compra     | compra                 |
| 3    | compra     | no compra              |
| 4    | compra     | no compra              |
| 5    | compra     | no compra              |
| 6    | no compra  | compra                 |
| 7    | no compra  | no compra              |
| 8    | no compra  | no compra              |

**Totales:**
- TP = 2 (1, 2)
- FN = 3 (3, 4, 5)
- FP = 1 (6)
- TN = 2 (7, 8)

---

#### Nuevos cálculos:

- `Precision = 2 / (2 + 1) = 0.666`
- `Recall = 2 / (2 + 3) = 0.400`
- `F1 Score = 2 * (0.666 * 0.4) / (0.666 + 0.4) ≈ 0.5`
- `Accuracy = (2 + 2) / 8 = 0.5`

---

**Interpretación:**
- `Precision` mejora: de 3 predicciones de "compra", 2 fueron correctas.
- `Recall` baja: el modelo detecta sólo 2 de los 5 casos reales de "compra".
- `F1` refleja este desequilibrio.


---

### 4. Comparación intuitiva

| Métrica   | Enfocada en...                           | Pregunta que responde                                  |
|-----------|-------------------------------------------|--------------------------------------------------------|
| Precision | Predicciones del modelo                   | ¿Cuántas veces acerté cuando **dije que sí**?          |
| Recall    | Casos reales de la clase                  | ¿Cuántas veces **dije que sí cuando debía hacerlo**?   |

---

### 5. Volvemos a nuestras métricas reales

| Métrica     | Valor   | Qué indica                                                      |
|-------------|---------|------------------------------------------------------------------|
| Accuracy    | 0.4070  | El 40.7% del total fue correctamente clasificado.               |
| Precision   | 0.2883  | Sólo el 28.83% de las predicciones positivas fueron correctas.  |
| Recall      | 0.4070  | Se identificó el 40.7% de los positivos reales.                 |
| F1 Score    | 0.3244  | Equilibrio bajo entre precisión y recall; desempeño pobre.      |




---

#### Desempeño por clase (`Classification Report`)

| Clase        | Precision | Recall | F1-score | Support |
|--------------|-----------|--------|----------|---------|
| mas_de_una   | 0.43      | 0.78   | 0.56     | 843     |
| nada         | 0.00      | 0.00   | 0.00     | 509     |
| una          | 0.33      | 0.24   | 0.28     | 648     |

- El modelo **predice razonablemente bien** la clase `mas_de_una` (F1 = 0.56).
- **No clasifica correctamente** ningún caso de `nada` (precision y recall = 0.00).
- La clase `una` es predicha con dificultad (F1 = 0.28).

---

#### Promedios

| Tipo de promedio | Precision | Recall | F1-score |
|------------------|-----------|--------|----------|
| Macro avg        | 0.25      | 0.34   | 0.28     |
| Weighted avg     | 0.29      | 0.41   | 0.32     |

- **Macro promedio** penaliza la falta de desempeño en `nada` y `una`.
- **Promedio ponderado** mejora ligeramente por el buen desempeño en `mas_de_una`.

---

#### Matriz de Confusión

| Real \ Predicha | mas_de_una | nada | una |
|-----------------|-------------|------|-----|
| mas_de_una      | 658         | 1    | 184 |
| nada            | 373         | 0    | 136 |
| una             | 492         | 0    | 156 |

- El modelo **predice masivamente como `mas_de_una`**, incluso cuando la clase real es otra.
- Para `nada`, **no hay ningún acierto**.
- Para `una`, los errores son frecuentes y tienden a ser confundidos con `mas_de_una`.

---

### Entonces

El modelo está sesgado hacia la clase mayoritaria (`mas_de_una`), logrando buena cobertura de esa clase pero **a costa de ignorar por completo `nada`** y clasificando mal `una`. El `accuracy` se mantiene razonable sólo por la desproporción de clases, pero las métricas por clase revelan un desempeño pobre en términos generales.

---

### 6. Support: ¿Importa el tamaño de cada clase?

Sí. Para interpretar bien **precision** y **recall**, es clave tener en cuenta **cuántos casos hay en cada clase**. Esto se llama `support`.

#### En nuestro conjunto de test:

| Clase        | Casos reales (`support`) | % del sample |
|--------------|--------------------------|--------------|
| mas_de_una   | 843                      | 42.2%        |
| una          | 648                      | 32.4%        |
| nada         | 509                      | 25.5%        |

- La clase `mas_de_una` tiene mayor representación (42% del test set).
- La clase `nada` es la más pequeña (25%).

Esto **condiciona el aprendizaje del modelo**:  
- El modelo tiende a **optimizar su desempeño en las clases más frecuentes**.
- En clases con poco `support`, puede fallar más y aun así mantener un buen `accuracy` general.
- Por eso **es crucial mirar precision y recall por clase**, no sólo el promedio.

---

### ¿Qué puede pasar?

| Clase poco frecuente (bajo support) | Riesgo común                         |
|-------------------------------------|--------------------------------------|
| `nada`                              | Recall muy bajo; el modelo la ignora |
| `una`                               | Precision baja; muchas falsas alarmas|

Esto también es una **señal de potencial overfitting o underfitting (inframodelado)**:  
- Si el modelo **memoriza casos raros**, cae en overfitting.  
- Si **no logra detectar clases pequeñas**, cae en underfitting y queda subajustado.


### Pero cómo sabemos si estos porcentajes están bien? 

---

In [ ]:
# entropía y support, medimos la cantidad de información heterogénea que aporta cada clase

# Calcular soporte desde y_test
support = y_test.value_counts()

#  proporciones
total = support.sum()
proportions = support / total

#  entropía de Shannon
entropy_support = -np.sum(proportions * np.log2(proportions))

print(f"\n--- Entropía del soporte observado: {entropy_support:.4f} bits ---")
print("Umbrales orientativos:")
print("  Entropía < 1.0  → distribución muy desbalanceada")
print("  Entropía ≈ 1.5  → balance moderado entre clases")
print("  Entropía > 1.5  → distribución altamente balanceada (máximo ≈ log2(N clases))")

### Interpretación de la entropía del soporte

La entropía calculada es **1.5546 bits**, lo cual está **ligeramente por encima del umbral de 1.5**.

Esto indica que:

- La distribución entre las tres clases (`nada`, `una`, `mas_de_una`) es **balanceada**.
- No hay una dominancia extrema de una clase sobre las otras.
- La entropía máxima posible para 3 clases sería `log2(3) ≈ 1.5849`, lo que marca un escenario perfectamente balanceado (33.3% cada clase). Dado que la entropía observada está muy cerca de ese valor, **el soporte está bastante equilibrado**.

**Conclusión**:  
El modelo no enfrenta un desbalance severo en la variable objetivo. Por lo tanto, si hay problemas de recall o precisión, **no se deben a una distribución desbalanceada de clases**, sino a **otras limitaciones del modelo** (como **profundidad**, ruido o segmentaciones poco efectivas).

# Sobre overfitting y underfitting

---

## Tamaño de clase (n) y riesgo de overfitting

- El **overfitting se agrava cuando hay clases con pocos casos**.
- Si una clase tiene muy pocos ejemplos (`n` bajo o `< 1%` del total), el árbol puede crear **reglas muy específicas que no se replican**.
- Esto lleva a:
  - **Recall bajo**: el modelo rara vez acierta esa clase.
  - **Precision falsa**: acierta en entrenamiento, pero nunca en test.
  - **Ramas inútiles**: hojas creadas para explicar 1 o 2 casos.
- **Señal crítica**: si una clase tiene `n < 1%-5%` del total y aún así genera ramas profundas, es síntoma fuerte de overfitting.


## Underfitting 

	•	Underfitting = reglas demasiado generales, sin capturar estructura real. Falta depth y precisión. 

Esto también puede generar recall bajo y mala precisión. Pero con buen support!! 

- Entonces: **Underfitting** ocurre cuando el modelo aprende **reglas demasiado generales** y no logra capturar **la estructura real de los datos**.
- Se manifiesta típicamente por:
  - Baja `accuracy` general.
  - `Recall` y `precision` bajos para todas las clases.
  - Árboles **poco profundos**, con pocas ramas y divisiones.

---

### Importante: el support es clave

- A diferencia del overfitting, **el underfitting puede ocurrir incluso con clases bien distribuidas (alta entropia)**.
- Es decir, el modelo **puede tener buen `support`** (entropía alta y clases equilibradas), pero aún así fallar por no aprender suficientemente.
- El problema no es la distribución, sino la **falta de complejidad del modelo** (poca profundidad, pocas divisiones).

---

### En resumen:

| Causa               | Resultado típico                    | Support  |
|---------------------|-------------------------------------|----------|
| Overfitting         | Buen desempeño en train, pobre en test | Malo / desequilibrado |
| Underfitting        | Mal desempeño en train y test       | Bueno / balanceado     |

> **El `support` ayuda a descartar causas estructurales. Si está bien balanceado, pero el modelo falla, el problema no es la distribución sino la capacidad del árbol.**
    
    El saldo se resume en el depth y sus dimensiones






# Sobre el depth

#### Entonces el depth óptimo resulta clave para que el modelo no haga ni overfitting ni underfitting 

In [ ]:

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, f1_score, accuracy_score
)

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")
df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

resultados = []

for d in range(1, 51):
    clf = DecisionTreeClassifier(max_depth=d, criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    resultados.append({
        'depth': d,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision_weighted': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall_weighted': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1_weighted': f1_score(y_test, y_pred, average='weighted', zero_division=0)
    })

df_resultados = pd.DataFrame(resultados)
depth_optimo = df_resultados.loc[df_resultados['f1_weighted'].idxmax(), 'depth']

print("\n=== Métricas por profundidad ===")
print("\n=== Descripción de métricas (promedio ponderado por clase) ===")
print("- accuracy: proporción total de aciertos del modelo sobre todos los casos.")
print("           → Esperable: cuanto más alto, mejor. Ideal > 0.70")

print("- precision_weighted: precisión promedio de las clases, ponderada por su frecuencia.")
print("           → Esperable: alto (algo tipo > 0.60), indica cuántas falsas predicciones positivas se detectaron. Interpretación depende de criticidad de la clase")
print("           → al subir, baja los falsos positivos (FP)")

print("- recall_weighted: recall promedio de las clases, ponderado por su frecuencia.")
print("           → Esperable: alto (algo tipo > 0.60-0.70), indica cuántos positivos reales fueron detectados.")
print("           → al subir, baja los falsos negativos (FN)")

print("- f1_weighted: media armónica entre precision y recall, ponderada por la frecuencia de clase.")
print("           → Esperable: valor equilibrado, algo tipo > 0.50-0.60. Penaliza si alguna métrica para una clase(s) es muy baja.")
print("           → baja el error de generalización (EGE) si tiene un valor medio")

print("--------------------------------\n\n")


print(df_resultados.round(4).to_string(index=False))
print(f"\nProfundidad óptima: {depth_optimo}")
# como ejercicio, a estos indicadores, se podrían agregar deltas para ver el trade-off entre depths y los puntos de saturación

### Análisis del desempeño del modelo de clasificación por profundidad

La evaluación del árbol de decisión muestra tres fases bien diferenciadas:

1. **Fase trivial (depth 1 a 3)**
El modelo predice casi exclusivamente la clase mayoritaria. Esto se refleja en una accuracy elevada pero engañosa (0.4215), mientras que la precision_weighted y el f1_weighted permanecen muy bajos (≈0.25). No hay aprendizaje efectivo.

2. **Fase de ganancia marginal (depth 4 a 20)**
Entre depth = 4 y depth = 20, el árbol comienza a aprender reglas útiles. La precision y el f1 ponderados mejoran progresivamente. Sin embargo, hay cierta inestabilidad, con aumentos y retrocesos leves que muestran que el modelo todavía oscila entre generalizar y sobreajustar. El f1_weighted se estabiliza alrededor de 0.36 sin un máximo claro hasta ese punto.

3. **Fase de máximo desempeño (depth 33)**
El modelo alcanza en depth = 33 sus mejores métricas conjuntas (accuracy: 0.3665, precision_weighted: 0.3663, f1_weighted: 0.3664). Sin embargo, ya en los niveles previos (≈28–32) las métricas habían comenzado a converger. Este punto representa el equilibrio máximo entre precisión y profundidad antes de que se manifiesten síntomas de sobreajuste. No se trata de un salto significativo, sino de una meseta breve de desempeño máximo.

    Si hay muchas menos instancias de una clase (como “una”, o "nada"), el modelo tiene menos datos para aprenderla. El valor moderado de F1 (≈0.35) no refleja necesariamente mal desempeño, sino las limitaciones impuestas por el desbalance  —tal como lo indica el support, donde hay 25.5% de “nada”—. Esta asimetría tiende a penalizar el rendimiento promedio ponderado de las clases minoritarias, y por lo tanto no deja que el f1 tenga mejor desempeño.

4. **Overfitting estructural**
A partir de depth = 39, todas las métricas permanecen idénticas hasta depth = 50, a pesar de que el árbol continúa creciendo en depth. Esta estabilización artificial indica que las nuevas divisiones no aportan ningún poder predictivo adicional, y que el modelo ya ha agotado la capacidad de generalizar del conjunto de entrenamiento. Es una señal clara de sobreajuste estructural, no visible por caída de métricas, sino por su congelamiento.


>    El modelo logra su mejor desempeño global a depth = 33, y no presenta mejoras más allá de ese punto. A partir de allí, se mantiene constante sin caer en sobreajuste medible por estas métricas. Este comportamiento sugiere que:

        •	El árbol necesita profundidad elevada para captar las interacciones necesarias entre variables en un problema multiclase desbalanceado.
        •	No hay señales de sobreajuste severo hasta depth = 50.
        •	El uso de f1_weighted como criterio es crucial para evitar que el modelo simplemente optimice por la clase mayoritaria.


# Moraleja: 

### Un árbol demasiado superficial subrepresenta el fenómeno (underfitting).
### 	•	Un árbol demasiado profundo comienza a memorizar ruido o combinaciones irrelevantes (overfitting).
### 	•	El punto óptimo suele encontrarse justo antes de que el modelo empiece a degradar sus métricas por complejidad excesiva.



---



# Diferencias entre Regression Trees y Clasifier Trees

### Diferencias en la profundidad óptima entre árboles de regresión y clasificación multiclase

La profundidad óptima de un árbol de decisión varía según el tipo de tarea y las características del dataset. En general, los árboles de regresión (`DecisionTreeRegressor`) requieren menos profundidad que los árboles de clasificación multiclase (`DecisionTreeClassifier`). A continuación se detallan las razones, incluyendo la interacción con el tamaño muestral (`n`).

---

#### Árboles de regresión: ¿Por qué menor profundidad? (si recuerdan, el óptimo era 4!)

1. **Variable objetivo continua**  
   - Las diferencias entre observaciones se capturan desde niveles tempranos.  
   - Las divisiones son eficaces rápidamente al reducir la dispersión.  
   - No es necesario separar clases distintas, lo que simplifica la estructura.

2. **Reducción de la varianza por nodo e intranodo**  
   - Cada split minimiza la varianza dentro de los nodos hijos (reduce MSE).  
   - La mejora se vuelve marginal tras pocos niveles.  
   - Sobreajuste común más allá de 4–6 niveles.

3. **Alta cardinalidad en `y`**  
   - `y` puede tener miles de valores únicos (montos de ventas).  
   - El árbol puede sobreajustar con facilidad al intentar predecir valores exactos.  
   - La complejidad aumenta exponencialmente con profundidad.

4. **Sensibilidad al tamaño muestral (`n`)**  
   - Con `n` reducido, la profundidad elevada genera nodos vacíos o con bajo n.  
   - Esto introduce ruido, inestabilidad y deterioro del arbol.
   - El tamaño muestral condiciona directamente la profundidad útil y la estabilidad estructural del árbol.


#### Clasificación multiclase: ¿Por qué mayor profundidad? (el óptimo de 33 depth!!!!)

1. **Separación de clases discretas**  
   - Las reglas deben aislar clases distintas que pueden estar solapadas.  
   - Requiere secuencias de condiciones más específicas.  
   - La estructura crece hasta que cada clase queda bien delimitada.

2. **Presencia de clases minoritarias (desbalance)**  
   - Las clases poco frecuentes emergen sólo con suficiente profundidad.  
   - El árbol necesita subdividir nodos mayoritarios para llegar a estas clases.  
   - La baja profundidad invisibiliza estas estructuras.

3. **Optimización del F1 ponderado**  
   - Esta métrica valora tanto la precisión como el recall, ponderados por frecuencia.  
   - Mejorar el recall en clases pequeñas suele exigir mayor complejidad.  
   - La profundidad permite lograr un mejor equilibrio entre cobertura y especificidad.

4. **Relación con el tamaño muestral (`n`)**  
   - Con `n` elevado, el árbol puede profundizar sin perder estabilidad.  
   - Las particiones profundas mantienen suficiente soporte estadístico por nodo.  
   - La penalización por complejidad es más lenta cuando `n` es grande.

---

Recalculamos con depth optimo antes de pasar a ensambling... 


In [ ]:
df = pd.read_csv("dataset_suscripciones.csv")

df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                            labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()

y = df['ventas_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(
    max_depth=33,
    criterion='entropy',
    random_state=42
)
clf.fit(X_train, y_train)

tree_text = export_text(clf, feature_names=list(X.columns))

view_dectree(
    tree_model=clf,
    features=list(X.columns),
    nombre_base='arbol_clasificacion_ventas',
    horizontal_spacing=1.0,
    vertical_spacing=1.2,
    font_size=16,
    flecha_grosor_factor=2,
    min_leaf_pct=0.01
)

print("\n=== Árbol categórico con métricas ===\n")
print(view_text_cattree_detailed(clf, X.columns))

print("\n=== EVALUACIÓN DEL MODELO DE CLASIFICACIÓN ===\n")
y_pred = clf.predict(X_test)
y_test_str = y_test.astype(str)
y_pred_str = pd.Series(y_pred).astype(str)

acc = accuracy_score(y_test_str, y_pred_str)
prec = precision_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
rec = recall_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
f1 = f1_score(y_test_str, y_pred_str, average='weighted', zero_division=0)

print(f"Accuracy: {acc:.4f}")
print(f"Precision (weighted): {prec:.4f}")
print(f"Recall (weighted): {rec:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

print("\n=== DISTRIBUCIÓN DE CLASES ===\n")
print("\nClases predichas:")
print(pd.Series(y_pred).value_counts(normalize=True))

print("Atención!:")

print("Profundidad real del árbol:", clf.get_depth())
print("Cantidad de hojas:", clf.get_n_leaves())

### Si bien encontramos el depth en donde la ganancia no llega, el problema que nos vamos a enfrentar es que la legibilidad del árbol se ve severamente comprometida! 

#### Apliquemos técnicas de pre-prunning, para limpiar el árbol. 

#### Esto va a implicar una salida más limpia y una contención mayor de los niveles. 


In [ ]:


df = pd.read_csv("dataset_suscripciones.csv")

df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()

y = df['ventas_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del árbol con pre-pruning
# --- AJUSTE NUEVO: min_samples_leaf y min_samples_split en base al 2% del sample ---
min_leaf = int(len(X_train) * 0.02)  # mínimo 2% del sample por hoja
min_split = int(len(X_train) * 0.05)  # mínimo 5% del sample para split

clf = DecisionTreeClassifier(
    max_depth=33,
    criterion='entropy',
    min_samples_leaf=min_leaf,        # nuevo: controlo hojas
    min_samples_split=min_split,      # nuevo: controlo divisiones en ramas 
    random_state=42
)
clf.fit(X_train, y_train)

tree_text = export_text(clf, feature_names=list(X.columns))

view_dectree(
    tree_model=clf,
    features=list(X.columns),
    nombre_base='arbol_clasificacion_ventas',
    horizontal_spacing=1.0,
    vertical_spacing=1.2,
    font_size=16,
    flecha_grosor_factor=2,
    min_leaf_pct=0.01
)

print("\n=== Árbol categórico con métricas ===\n")
print(view_text_cattree_detailed(clf, X.columns))

# Evaluación
print("\n=== EVALUACIÓN DEL MODELO DE CLASIFICACIÓN ===\n")
y_pred = clf.predict(X_test)
y_test_str = y_test.astype(str)
y_pred_str = pd.Series(y_pred).astype(str)

acc = accuracy_score(y_test_str, y_pred_str)
prec = precision_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
rec = recall_score(y_test_str, y_pred_str, average='weighted', zero_division=0)
f1 = f1_score(y_test_str, y_pred_str, average='weighted', zero_division=0)

print(f"Accuracy: {acc:.4f}")
print(f"Precision (weighted): {prec:.4f}")
print(f"Recall (weighted): {rec:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

# Distribución de clases
print("\n=== DISTRIBUCIÓN DE CLASES ===\n")
print("\nClases predichas:")
print(pd.Series(y_pred).value_counts(normalize=True))

print("\n\nAtención!:")

print("Profundidad real del árbol:", clf.get_depth())
print("Cantidad de hojas:", clf.get_n_leaves())

<!-- comparativa_arboles_recomendaciones_detalladas_v5.html -->
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <title>Comparativa con Acciones Estratégicas</title>
  <style>
    body {
      font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
      background-color: #f9f9f9;
      color: black;
      padding: 40px;
      max-width: 1080px;
      margin: auto;
      line-height: 1.6;
    }

    h1 {
      color: #003366;
      border-bottom: 3px solid #003366;
      padding-bottom: 8px;
      margin-bottom: 30px;
    }

    table {
      border-collapse: collapse;
      width: 100%;
      margin-top: 30px;
      font-size: 0.95em;
      color: black;
    }

    th, td {
      border: 1px solid #ccc;
      padding: 10px 12px;
      text-align: left;
      vertical-align: top;
      background-color: white;
      color: black;
    }

    th {
      background-color: #006699;
      color: white;
      font-weight: bold;
    }

    caption {
      text-align: left;
      font-size: 1.4em;
      font-weight: bold;
      margin-bottom: 10px;
      color: #003366;
    }

    code {
      background-color: #e8e8e8;
      padding: 2px 5px;
      border-radius: 3px;
      font-family: monospace;
      color: black;
    }

    .amarillo td {
      background-color: #fff8dc;
    }

    .verde td {
      background-color: #e0f2e0;
    }
  </style>
</head>
<body>

<h1>Comparativa de árboles y acciones recomendadas</h1>

<table>
  <caption>Variables, estructura y recomendaciones de acción</caption>
  <thead>
    <tr>
      <th>Variable</th>
      <th>Árbol Original (depth=4)</th>
      <th>Árbol Ajustado (depth=33 (depth 9+preprune))</th>
      <th>Recomendación Estratégica</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><code>paginas_visitadas</code></td>
      <td>Raíz</td>
      <td>Raíz + subramas amplias</td>
      <td>Segmentar desde el inicio por intensidad de exploración. Personalizar desde el 2–3er click. Poca navegación → ofrecer CTA directos (e.g., botón de compra destacado).</td>
    </tr>
    <tr>
      <td><code>codigo_promocional</code></td>
      <td>Segundo nodo</td>
      <td>Segundo nodo + con interacciones</td>
      <td>Activar descuentos condicionados a navegación previa y rating. Evitar mostrar códigos a <u>usuarios fríos</u> (sin navegación ni interacción).</td>
    </tr>
    <tr class="amarillo">
      <td><code>rating_promedio</code></td>
      <td>Intermedio</td>
      <td>Condición clave en múltiples ramas</td>
      <td>Usuarios que navegan productos con bajo rating: mostrar prueba social (opiniones, garantías, devoluciones). Rating alto: usar escasez o urgencia ("últimos disponibles").</td>
    </tr>
    <tr class="amarillo">
      <td><code>logueado</code></td>
      <td>Ausente</td>
      <td>Condición estructural</td>
      <td>Usuarios logueados tienen comportamiento más predecible: mostrar precios completos sin descuentos. Usuarios anónimos: usar prueba social y CTA directos.</td>
    </tr>
    <tr class="amarillo">
      <td><code>clicks</code></td>
      <td>Ausente</td>
      <td>Intermedio</td>
      <td>Muchos clicks → perfil comparador → se puede ofrecer precio más alto con buenos argumentos (reputación, garantía). Pocos clicks → usar ofertas urgentes.</td>
    </tr>
    <tr class="amarillo">
      <td><code>edad</code></td>
      <td>Marginal</td>
      <td>Presente en ramas de decisión final</td>
      <td>Ofertas adaptadas por rango etario. Jóvenes: promociones y estética visual. Mayores: destacar soporte, claridad y confianza.</td>
    </tr>
    <tr class="amarillo">
      <td><code>tiempo_browsing</code></td>
      <td>Ausente</td>
      <td>Condición específica con <code>precio</code></td>
      <td>Usuarios que navegan mucho sin decidir: lanzar pop-up con descuento por tiempo limitado o CTA fuerte (Call To Action: "Aprovechá antes de que se agote").</td>
    </tr>
    <tr class="verde">
      <td><code>precio</code></td>
      <td>Intermedio</td>
      <td>Condición frecuente</td>
      <td>
        Diseñar precios dinámicos. Si el usuario pasa por muchas condiciones del árbol (rama profunda), es más probable que tolere precios altos si el valor percibido es alto. <br><br>
        Detectar usuarios que cumplen ≥4 condiciones sucesivas del árbol (ej. muchas páginas, logueo, buen rating, múltiples clicks, edad alta). <br>
        Considerarlos usuarios de alta intención:<br>
        – No ofrecer descuentos automáticos<br>
        – Usar CTA de escasez o urgencia ("últimos lugares", "comprado hoy por 28 personas")<br>
        – Elevar ligeramente el precio si el rating es alto<br>
        – Priorizar para recomendaciones premium o up-selling
      </td>
    </tr>
    <tr>
      <td><code>descuento_pct</code></td>
      <td>Secundaria</td>
      <td>Condición intermedia</td>
      <td>Usar descuentos como incentivo sólo en usuarios fríos o en productos con bajo rating y usuarios jóvenes. Evitar ofrecerlos a quienes ya muestran intención alta.</td>
    </tr>
    <tr class="amarillo">
      <td><code>es_domingo</code></td>
      <td>Ausente</td>
      <td>Condición puntual</td>
      <td>Testear campañas especiales de fin de semana. En usuarios anónimos o logueados con promoción, se potencia el efecto.</td>
    </tr>
    <tr class="amarillo">
      <td><code>es_feriado</code></td>
      <td>Ausente</td>
      <td>Ausente</td>
      <td>No se detectan patrones claros. Requiere más datos específicos de días feriados para tomar decisiones.</td>
    </tr>
  </tbody>
</table>

</body>
</html>

# Métodos de Ensembling

**Objetivo:** mejorar la robustez y precisión de los modelos basados en árboles de decisión.

### Estrategias principales:

---

1. **Vía Modelos o algoritmos**  
   - Combinación de algoritmos diferentes (p. ej., árboles de regresión y de clasificación)
   - Combinación de especificaciones de algoritmos (p. ej., depth, pruning)

---

2. **Training Set**  
   - Variación del training set, nuevo training set, diferente training set
   - Uso de conjuntos de datos distintos  
   - Particiones aleatorias del mismo dataset (submuestreo --> repeated subsampling)

---

3. **Bagging (Bootstrap Aggregating)**  
   - Entrena múltiples árboles sobre subconjuntos diferentes de datos
     - Se generan múltiples subconjuntos del dataset original mediante *bootstrap* (muestreo con reemplazo)  
   - Combina predicciones para reducir la varianza y se combinan los resultados (promedio, votación)
   - Ejemplo: Random Forests

---

4. **Boosting**  
   - Técnica secuencial: los modelos se entrenan uno tras otro  
   - Cada nuevo modelo se enfoca en evitar o mitigar los errores cometidos por el modelo anterior  
   - Se ajusta el peso de las observaciones: mayor peso a los casos mal predichos (Error-driven learning; Sequential correction on Hardness-weighted sampling)
   - Al final, se combinan todos los modelos con un sistema de ponderación (en base al learning rate, que puede decrecer) 
   - Ejemplo: XGBoost (eXtreme Gradient Boosting)  
   - Ventaja: reduce el sesgo del modelo base, mejora la precisión en datasets con relaciones complejas o ruido moderado  
   - Requiere regularización y control ESTRICTO de sobreajuste debido a su potencia


## Repeated Subsampling

### Finalidad del procedimiento

Este procedimiento tiene como objetivo evaluar la estabilidad y consistencia de un árbol de decisión con profundidad previamente optimizada (depth = 16), aplicado sobre distintas particiones aleatorias del dataset. Permite observar cómo varía el rendimiento general del modelo ante pequeños cambios en la muestra de entrenamiento.

### Descripción del proceso

1. **Identificación del depth óptimo**  
   Previo al muestreo repetido, se identificó la profundidad óptima del árbol de decisión ejecutando una validación sobre una única partición del dataset (70% entrenamiento, 30% test).  
   Para ello, se evaluó el desempeño de árboles con distintas profundidades (`max_depth` entre 1 y 50), utilizando el F1 ponderado como criterio de optimización.  
   La profundidad que maximizó este indicador fue seleccionada como valor óptimo (depth = 16).

2. **Submuestreo aleatorio del dataset**  
   En cada iteración, se extrae una submuestra aleatoria del 70% del dataset original para ser usada como conjunto de entrenamiento. El 30% restante no se utiliza en esta etapa. Cada submuestreo se genera con una semilla distinta.

3. **Entrenamiento con profundidad fija y evaluación sobre el dataset completo**  
   Para cada submuestra, se entrena un árbol de decisión (`DecisionTreeClassifier`) con profundidad fija en 16. Luego se evalúa su rendimiento prediciendo sobre el dataset completo (train + test), calculando los siguientes indicadores:

   - Precisión ponderada (`precision_weighted`)  
   - Recall ponderado (`recall_weighted`)  
   - F1 ponderado (`f1_weighted`)  
   - Accuracy total (`accuracy`)

Este procedimiento permite verificar si el modelo produce resultados consistentes y si su estructura interna es estable frente a distintos subconjuntos del dataset, lo cual es clave antes de considerar esquemas de ensamblado más complejos.

In [ ]:
df = pd.read_csv("dataset_suscripciones.csv")

df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                          labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

# División 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

# Loop para Evaluar el árbol con un rango de distintas profundidades (1 a 50)
# buscamos el depth que maximiza el f1_weighted
resultados = []

for d in range(1, 51):
    clf = DecisionTreeClassifier(max_depth=d, criterion='entropy', random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    resultados.append({
        'depth': d,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision_weighted': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall_weighted': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1_weighted': f1_score(y_test, y_pred, average='weighted', zero_division=0)
    })

df_resultados = pd.DataFrame(resultados)
depth_optimo = df_resultados.loc[df_resultados['f1_weighted'].idxmax(), 'depth']

print(df_resultados.round(4).to_string(index=False))
print(f"\nProfundidad óptima: {depth_optimo}")

In [ ]:


# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")

df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

# Evaluación con repeated subsampling 
resultados_rf = []
n_iteraciones = 50
random_states = np.random.randint(0, 10000, size=n_iteraciones) # esto es para que cada split 70/30 sea distinto

for seed in random_states:
    # repite División 70/30, cada vuelta toma observaciones distintas. La división es siempre 70/30
    X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y, train_size=0.7, random_state=seed)

    model = RandomForestClassifier(n_estimators=100, max_depth=16, random_state=seed)
    model.fit(X_train_sub, y_train_sub)
    y_pred = model.predict(X_test_sub)

    resultados_rf.append({
        'seed': seed,
        'accuracy': accuracy_score(y_test_sub, y_pred),
        'precision_weighted': precision_score(y_test_sub, y_pred, average='weighted', zero_division=0),
        'recall_weighted': recall_score(y_test_sub, y_pred, average='weighted', zero_division=0),
        'f1_weighted': f1_score(y_test_sub, y_pred, average='weighted', zero_division=0),
        'n_trees': model.n_estimators,
        'avg_depth': np.mean([tree.get_depth() for tree in model.estimators_]),
        'report': classification_report(y_test_sub, y_pred, zero_division=0)
    })

df_resultados_rs = pd.DataFrame(resultados_rf)
print("=== RESULTADOS REPEATED SUBSAMPLE ===")
display(df_resultados_rs.drop(columns='report').round(4))
print("\n--- Descripción estadística ---\n")
display(df_resultados_rs.drop(columns='report').describe().round(4))

print("\n=== COMPARACIÓN CON ÁRBOL DE DECISIÓN ===")
print("Repeated Subsample:")
print(f"Accuracy: {df_resultados_rs['accuracy'].mean():.4f}")
print(f"Precision (weighted): {df_resultados_rs['precision_weighted'].mean():.4f}")
print(f"Recall (weighted): {df_resultados_rs['recall_weighted'].mean():.4f}")
print(f"F1 Score (weighted): {df_resultados_rs['f1_weighted'].mean():.4f}")

print("\nÁrbol de Decisión:")
print(f"Accuracy: {0.4205:.4f}")
print(f"Precision (weighted): {0.3840:.4f}")
print(f"Recall (weighted): {0.4205:.4f}")
print(f"F1 Score (weighted): {0.3518:.4f}")

## ¿Qué puede observarse del modelo aunque no se visualicen los paths?

Aunque los modelos más complejos (como Random Forest o árboles con muchas iteraciones sobre subconjuntos de datos) no ofrecen reglas de decisión explícitas y trazables, permiten evaluar distintos aspectos clave de su rendimiento y utilidad. A continuación se resumen los principales:

### 1. Predicciones confiables sobre nuevos casos
Incluso sin conocer la lógica exacta detrás de cada decisión, es posible utilizar el modelo para clasificar observaciones futuras con mayor precisión que modelos más simples. Esto es particularmente útil cuando el objetivo es la performance predictiva y no la interpretabilidad.

### 2. Importancia de variables
El modelo puede proporcionar métricas de importancia para cada variable de entrada, ya sea a través de su contribución a la reducción de impureza o mediante métodos más robustos como la permutación. Esto permite identificar qué atributos influyen más en las decisiones del modelo, aunque no se conozca cómo interactúan entre sí.

### 3. Estabilidad del rendimiento (robustez)
Mediante técnicas como el submuestreo repetido (*repeated subsampling*), se puede observar cómo varía el rendimiento del modelo ante cambios en la muestra de entrenamiento. Si los resultados se mantienen estables, el modelo se considera robusto y confiable frente a fluctuaciones en los datos.

### 4. Indicadores de balance entre clases
El análisis de métricas como *precision*, *recall* y *F1-score* permite detectar si el modelo está sesgado hacia clases mayoritarias o si es capaz de capturar con efectividad eventos poco frecuentes. Esto es clave para problemas con clases desbalanceadas o con mayor costo de error en ciertas categorías.

En conjunto, estos elementos permiten validar la capacidad del modelo para generalizar, evaluar su confiabilidad estadística y entender parcialmente su comportamiento, incluso en ausencia de una representación visual directa de sus reglas internas.

#### Es esperable que en estas técnicas de bagging (como subsampling) encontremos resultados **más robustos** pero menos precisos. Es decir, los indicadores se deterioran debido a que intruducimos "ruido" aleatorio en cada selección de datos. 

---

# Random Forests

In [ ]:
print("""
                          +--------------------+
                          |   Random Forest    |
                          +--------------------+
                                   |
     +-----------------------------+-----------------------------+
     |                             |                             |
+------------+             +------------+               +------------+
|  Árbol #1  |             |  Árbol #2  |       ...     | Árbol #100 | (n_estimators)
+------------+             +------------+               +------------+
     |                             |                             |
 Predicción y₁             Predicción y₂               Predicción y₁₀₀
     \\                             |                             /
      \\___________________________ | ___________________________/
                                   ↓
                      Promedio de todas las predicciones
                                   ↓
                     → Predicción final del modelo (ŷ)
""")



---

### Diferencias entre Submuestreo Aleatorio y Bootstrap en Ensembles

Al construir modelos de tipo ensemble, como los árboles de decisión múltiples, es importante distinguir entre dos estrategias comunes para generar conjuntos de entrenamiento: el submuestreo aleatorio sin reemplazo y el muestreo bootstrap (con reemplazo). Aunque ambos parten del mismo dataset original, sus objetivos y mecanismos son distintos.

---

#### 1. Submuestreo Aleatorio (Repeated Subsampling / Repeated Subtraining)

- Se generan múltiples subconjuntos del dataset **sin reemplazo**, es decir, cada observación aparece una sola vez por subconjunto.
- Se utilizan diferentes particiones aleatorias para simular distintos escenarios de entrenamiento.
- Su propósito principal es **evaluar la robustez del modelo**, no construir un ensemble final.
- No produce un modelo único consolidado, sino una distribución de métricas a partir de múltiples entrenamientos independientes.
- Es una técnica útil para **validación repetida**, pero no reduce el error del modelo base.

---

#### 2. Bootstrap (usado en Bagging y Random Forest) 

- Se generan subconjuntos de entrenamiento **con reemplazo**: algunas observaciones pueden aparecer múltiples veces y otras no estar presentes.
- Cada subconjunto cubre en promedio el 63.2% de los casos únicos del dataset original.
- Cada modelo (por ejemplo, cada árbol en un Random Forest) es entrenado sobre una muestra distinta y potencialmente sesgada.
- La agregación posterior (por mayoría o promedio) **reduce la varianza del modelo final**, mejorando su estabilidad y generalización.
- Es la base de técnicas de ensamble como Bagging y Random Forest.

---

#### Comparación Sintética

| Característica                     | Submuestreo (sin reemplazo)           | Bootstrap (con reemplazo)         |
|------------------------------------|----------------------------------------|-----------------------------------|
| Reemplazo                          | No                                     | Sí                                |
| Objetivo principal                 | Evaluación de estabilidad              | Reducción de varianza             |
| Construye ensemble final           | No                                     | Sí                                |
| Diversidad entre modelos           | Limitada                               | Alta                              |
| Tamaño de cada muestra             | < 100%, sin repetidos                  | ≈ 100%, con repetidos             |
| Cobertura del dataset original     | Parcial                                | Total, pero heterogénea           |

---

El **submuestreo aleatorio** se utiliza para verificar si un modelo es sensible a pequeñas variaciones del conjunto de entrenamiento, mientras que el **bootstrap** es una estrategia explícita de mejora del modelo a través de agregación de múltiples predictores entrenados en datos modificados mediante muestreo con reemplazo.


----


### ¿Qué hace un Random Forest?

**Random Forest** es un algoritmo de aprendizaje supervisado que construye múltiples árboles de decisión para resolver problemas de clasificación o regresión. En lugar de apoyarse en un único árbol, crea un "bosque" de árboles que trabajan en conjunto para mejorar la precisión y la robustez del modelo.

#### ¿Cómo funciona?

1. **Bootstrap (muestreo aleatorio con reemplazo)**  
   A partir del conjunto de entrenamiento original, se generan múltiples subconjuntos aleatorios —con reemplazo— del mismo tamaño. A esto se lo llama **bootstrap sampling**. Cada árbol del bosque se entrena con uno de estos subconjuntos distintos.

2. **Entrenamiento de árboles independientes**  
   Cada árbol se entrena de forma independiente, con su subconjunto bootstrap. Durante la construcción, en cada división interna del árbol, el algoritmo selecciona una muestra aleatoria de las variables disponibles (no todas), lo que introduce mayor diversidad estructural entre los árboles.

3. **Agregación de resultados (votación)**  
   Una vez entrenado el bosque, cada árbol emite una predicción. En clasificación, el resultado final del Random Forest se obtiene por **votación mayoritaria**: la clase que recibe más votos entre los árboles es la predicción final.

#### ¿Por qué es eficaz?

- **Reduce el sobreajuste**: la variabilidad introducida por el bootstrap y la selección aleatoria de variables impide que los árboles se adapten demasiado al ruido.
- **Mejora la generalización**: al combinar muchos modelos débiles, se obtiene un modelo fuerte con menor varianza.
- **Es robusto a datos ruidosos y desbalanceados**, aunque en estos casos puede requerir ajustes adicionales (como balanceo de clases o ajuste de pesos).

---


---
El **rol central del Random Forest** no es necesariamente mejorar drásticamente la eficiencia predictiva frente al mejor árbol individual, sino **reducir el sobreajuste** y garantizar que los resultados sean **más estables y robustos** frente a nuevas muestras.

Al trabajar con múltiples árboles entrenados sobre diferentes subconjuntos aleatorios (bootstrap) y seleccionando aleatoriamente las variables en cada división, el modelo reduce la dependencia de particularidades del conjunto de entrenamiento. Esto produce una **mejora en la generalización**, incluso si no incrementa notablemente la precisión o el recall en un caso puntual.

En este sentido, la ganancia clave está en la **consistencia del modelo**, no tanto en una mejora automática y sustancial del rendimiento global. Esto es especialmente importante en contextos con ruido, alta dimensionalidad o cuando se requiere replicabilidad del desempeño en distintos cortes del mismo problema.

---



In [ ]:


import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")
df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

# División fija
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42
)

# Random Forest
min_leaf = int(len(X_train) * 0.01)
min_split = int(len(X_train) * 0.021)
max_depth = 40
seed = 42

modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=max_depth,
    min_samples_leaf=min_leaf,
    min_samples_split=min_split,
    random_state=seed,
    max_features=None
)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rec_rf = recall_score(y_test, y_pred_rf, average='weighted', zero_division=0)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

# Resultados
print("\n=== RANDOM FOREST (max_features=None) ===")
print(f"Accuracy: {acc_rf:.4f}")
print(f"Precision (weighted): {prec_rf:.4f}")
print(f"Recall (weighted): {rec_rf:.4f}")
print(f"F1 Score (weighted): {f1_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))


### ¿Qué hace este Random Forest en particular?

Este modelo se entrena para predecir una variable categórica con tres clases: `nada`, `una` y `mas_de_una`, que indican cuántas ventas genera un usuario. Utiliza 11 variables explicativas que reflejan el comportamiento de navegación, las promociones y las condiciones contextuales.

#### Configuración utilizada
- **100 árboles**
- **Máxima profundidad: 40**
- **Muestras mínimas por hoja y por división: 2% del conjunto de entrenamiento**
- **División fija entrenamiento/testeo (70% / 30%)**
- **Semilla aleatoria fija (reproducibilidad)**

#### Resultado observado
El modelo muestra un fuerte sesgo hacia la clase mayoritaria (`mas_de_una`), con alta capacidad de detección en esa clase pero bajo desempeño en las restantes (`una`, `nada`). Esto indica que, aunque el bosque es más robusto que un solo árbol, todavía sufre el impacto del desbalance de clases.

Este comportamiento es típico cuando no se aplican técnicas de balanceo ni ponderación en el entrenamiento, y debe corregirse si se requiere una cobertura uniforme en todas las categorías.

### ¿Qué es Feature Bagging?

**Feature bagging** es una técnica que consiste en seleccionar aleatoriamente un subconjunto de variables (features) en cada nodo de cada árbol dentro de un Random Forest. 

Entonces,  evaluar todas las variables disponibles para determinar el mejor punto de corte mediante la minimización de entropía, el modelo restringe la selección a un subconjunto aleatorio de ellas. Es decir, en cada nodo, el criterio de partición se aplica sobre un conjunto reducido de variables, no sobre el total: va a ser TOTAL - FEATURES EXTRAIDAS, que se definen aleatoriamente. 

Esto introduce diversidad adicional entre los árboles y reduce la correlación entre ellos, lo cual mejora la robustez del modelo y disminuye el riesgo de sobreajuste.

### Uso de `max_features="sqrt"`

En problemas de clasificación, el valor por defecto `max_features="sqrt"` indica que en cada nodo, el modelo seleccionará aleatoriamente una cantidad de variables igual a la raíz cuadrada del total de variables disponibles. Esto:
- Favorece la diversidad entre árboles.
- Introduce ruido controlado.
- Puede degradar el rendimiento sobre clases minoritarias si estas dependen de variables no seleccionadas.

Es una práctica común que mejora la generalización, especialmente en datasets grandes o con alta dimensionalidad.

In [ ]:


import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")
df['ventas_cat'] = pd.cut(df['ventas'], bins=[-1, 0, 1, df['ventas'].max()],
                        labels=['nada', 'una', 'mas_de_una'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_cat']

# División fija
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42
)

# Random Forest
min_leaf = int(len(X_train) * 0.02)
min_split = int(len(X_train) * 0.02)
max_depth = 40
seed = 42

modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=max_depth,
    min_samples_leaf=min_leaf,
    min_samples_split=min_split,
    random_state=seed,
    max_features='sqrt' 
)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rec_rf = recall_score(y_test, y_pred_rf, average='weighted', zero_division=0)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

# Resultados
print("\n=== RANDOM FOREST con Feature Bagging ===")
print(f"Accuracy: {acc_rf:.4f}")
print(f"Precision (weighted): {prec_rf:.4f}")
print(f"Recall (weighted): {rec_rf:.4f}")
print(f"F1 Score (weighted): {f1_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))

### Comparación entre Random Forest con y sin Feature Bagging

A continuación se presenta una tabla con indicadores comparativos clave entre dos versiones del modelo Random Forest:

| Métrica                | RF sin Feature Bagging | RF con Feature Bagging |
|------------------------|------------------------|------------------------|
| Accuracy               | 0.4177                 | 0.4267                 |
| Precision (weighted)   | 0.3487                 | 0.5377                 |
| Recall (weighted)      | 0.4177                 | 0.4267                 |
| F1 Score (weighted)    | 0.3064                 | 0.2880                 |
| F1 "mas_de_una"        | 0.58                   | 0.59                   |
| F1 "nada"              | 0.02                   | 0.00                   |
| F1 "una"               | 0.16                   | 0.10                   |
| F1 Macro               | 0.25                   | 0.23                   |
| F1 Weighted            | 0.31                   | 0.29                   |

**Interpretación general:**


- **Accuracy:** mejora levemente con feature bagging (0.4267 vs 0.4177), pero esta diferencia es marginal.
- **Precision (weighted):** mejora notable con feature bagging (0.5377), indicando que, cuando predice una clase, lo hace con mayor acierto.
- **Recall (weighted):** se mantiene prácticamente igual, sin mejoras significativas.
- **F1 Score (weighted):** es **más bajo con feature bagging**, lo que sugiere que el modelo es menos equilibrado entre precision y recall en general.
- **F1 Macro:** también **baja** con feature bagging. Esto significa que, en promedio, el desempeño por clase es peor, sobre todo en las minoritarias.
- **Desempeño por clase:** el modelo con feature bagging se concentra más en la clase mayoritaria ("mas_de_una") y **abandona completamente la clase "nada"**, cuyo F1 cae a 0.00.


> Feature bagging incrementa la precisión cuando el modelo acierta, pero **sacrifica la cobertura de clases minoritarias**, resultando en un modelo más sesgado hacia la clase mayoritaria. Aunque el `accuracy` mejora ligeramente, el balance general del modelo **empeora**, lo cual se refleja en los F1 promedio.

### **Aclaración sobre el desempeño CON Y SIN FEATURE BAGGING**

Aunque en general el uso de *feature bagging* en Random Forest **mejora la robustez** y la capacidad de **generalización** del modelo, su efecto NO SIEMPRE implica una mejora en las métricas.

En este caso, la activación de `max_features='sqrt'` mejora la `precision_weighted` y la `accuracy`, lo que sugiere UNA MEJOR DISCRIMINACIÓN sobre la clase MAYORITARIA. Sin embargo, el `F1 macro` y el `F1 weighted` empeoran, lo que INDICA UN MAYOR DESEQUILIBRIO en el rendimiento ENTRE clases.

Esto se debe a que, al limitar las variables consideradas por nodo, **el modelo pierde acceso a variables claves para clasificar correctamente las clases minoritarias**, que ya estaban en desventaja por su bajo soporte (n de clase). Por tanto, aunque el modelo es más robusto en términos de agregación, su capacidad para cubrir bien todas las clases disminuye.


---

### Interpretación del desempeño del modelo Random Forest

**1. Precisión global (`accuracy`)**
- El modelo acierta en el 42.67% de los casos, lo cual es apenas superior al azar si se considera el desbalance de clases.
- Sin embargo, la `precision` ponderada es alta (0.5377) - cuando el modelo predice una clase, suele acertarla.

**2. Desempeño por clase**
- **Clase "mas_de_una"**: excelente `recall` (0.95), pero con `precision` moderada (0.43). El modelo predice esta clase con frecuencia y acierta la mayoría, aunque incluye varios falsos positivos.
- **Clase "nada"**: `recall` de 0.00, lo que indica que el modelo prácticamente nunca la predice correctamente, a pesar de su `precision` artificialmente alta (1.00), producto de muy pocas o ninguna predicción efectiva.
- **Clase "una"**: pobre desempeño general, con `recall` 0.06 y `f1` muy bajo (0.10), lo que refleja dificultad en distinguir esta clase de las otras.

**3. Promedios**
- `Macro avg` (0.23 en F1) expone un desequilibrio grave: el modelo se concentra en una sola clase.
- `Weighted avg` (0.29 en F1) también es bajo, lo que evidencia que incluso ponderando por soporte, la calidad general es limitada.

**4. Entonces**
- El modelo Random Forest se especializa en predecir la clase **"mas_de_una"**, sacrificando completamente la capacidad de identificar correctamente las otras clases.
- Este comportamiento sugiere **un problema serio de desbalance de clases no corregido**, lo cual limita la utilidad práctica del modelo si se requiere cobertura razonable en las clases "una" y "nada".

---

# nota 1: depth en el forest

En RandomForestClassifier, si no se especifica el parámetro max_depth, cada árbol crece hasta que todos los nodos son puros o contienen menos que min_samples_split observaciones. Es decir:

- Por defecto, el depth de cada árbol en un random forest es ilimitado (max_depth=None).
- Esto implica que cada árbol individual puede sobreajustar, pero el conjunto (promediado o por votación) no lo hace fácilmente gracias a:
  - Bootstrapping (diferente subconjunto de datos por árbol)
  - Submuestreo de variables (cada split evalúa sólo un subconjunto aleatorio de variables)


### Nosotros ponemos depth 40 porque sabemos el optimal depth == 33
---


# nota 2: subsampling o forest?

### Round 2: Diferencias entre Repeated Subsampling y Random Forest

**1. Repeated Subsample**  
Este método consiste en entrenar el mismo modelo base (por ejemplo, un árbol de decisión) sobre múltiples *subconjuntos aleatorios* del dataset original (sin reemplazo), evaluando luego sobre el conjunto completo. Es útil para estimar la **robustez y estabilidad del modelo** ante distintas particiones, pero **no cambia la estructura del modelo** ni su forma de predecir.

- Su alta performance aquí refleja que **algunas particiones permiten encontrar árboles muy efectivos**, pero **no hay agregación de árboles**.
- Puede sobreestimar el rendimiento si las particiones favorecen patrones particulares o repiten estructuras fáciles de ajustar.

**2. Random Forest con Feature Bagging**  
Random Forest es un modelo *ensamblado*, que entrena múltiples árboles sobre subconjuntos *con reemplazo* (bootstrap) **y además** selecciona aleatoriamente un subconjunto de variables por nodo (feature bagging). Las predicciones se agregan mediante votación (clasificación) o promedio (regresión).

- Tiene más robustez general, pero **el feature bagging introduce ruido estructural** para evitar el sobreajuste.
- Su objetivo es minimizar la varianza a costa de un posible aumento del sesgo.
- Por esta razón, **la performance individual de cada árbol puede ser más baja**, pero el conjunto debería ser más confiable **en generalización**.


### A cuál le creo?

- **Repeated subsample** muestra qué tan bien puede funcionar un árbol en ciertos cortes de datos. No representa un modelo operativo, sino un diagnóstico.
- **Random Forest** es un modelo real, diseñado para funcionar con datos nuevos y evitar sobreajuste. Su performance puede ser más modesta, pero es **más estable y confiable** frente a nuevas observaciones.

**Entonces**: el rendimiento de repeated subsample no es comparable directamente con el de Random Forest. El primero es diagnóstico; el segundo, una solución robusta. En producción, **creerle al Random Forest**.

---

---

# Soluciones? 

### Estimemos el modelo con una clase dual en lugar de tres clases





In [ ]:


import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")

# Recodificación binaria de la variable objetivo
df['ventas_bin'] = np.where(df['ventas'] == 0, 'sin_ventas', 'con_ventas')  # si ventas == 0 → "sin_ventas", si no → "con_ventas"
# Alternativa usando pd.cut:
# df['ventas_bin'] = pd.cut(df['ventas'], bins=[-1, 0, df['ventas'].max()], labels=['sin_ventas', 'con_ventas'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_bin']

# División fija
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42
)

# Random Forest con feature bagging
min_leaf = int(len(X_train) * 0.02)
min_split = int(len(X_train) * 0.02)
max_depth = 40
seed = 42

modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=max_depth,
    min_samples_leaf=min_leaf,
    min_samples_split=min_split,
    random_state=seed,
    max_features='sqrt'
)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rec_rf = recall_score(y_test, y_pred_rf, average='weighted', zero_division=0)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

# Resultados
print("\n=== RANDOM FOREST BINARIO (nada vs algo) ===")
print(f"Accuracy: {acc_rf:.4f}")
print(f"Precision (weighted): {prec_rf:.4f}")
print(f"Recall (weighted): {rec_rf:.4f}")
print(f"F1 Score (weighted): {f1_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))

### Comparación de desempeño: Random Forest Multiclase vs Binario

| Métrica                          | Fórmula y Explicación                    | RF Multiclase (3 clases) | RF Binario (sin_ventas vs con_ventas) |
|----------------------------------|------------------------------------------|---------------------------|----------------------------------------|
| **Accuracy**                     | $\frac{TP + TN}{N}$     | 0.4267                    | 0.7530                                 |
|                                  | Proporción de predicciones correctas     |                           |                                        |
| **Precision (weighted)**         | $\frac{TP}{TP + FP}$                     | 0.5377                    | 0.5670                                 |
|                                  | Proporción de positivos correctos        |                           |                                        |
| **Recall (weighted)**            | $\frac{TP}{TP + FN}$                     | 0.4267                    | 0.7530                                 |
|                                  | Proporción de casos positivos efectivos (cuántas veces acerté ventas) |                           |                                        |
| **F1 Score (weighted)**          | $2 \times \frac{precision \times recall}{precision + recall}$ | 0.2880                    | 0.6469                                 |
|                                  | Media armónica entre precisión y recall  |                           |                                        |
| **F1 Score (macro)**             | Promedio de F1 por clase                 | 0.2300                    | 0.4300                                 |
|                                  | Equilibrio entre clases                  |                           |                                        |
| **Soporte clase minoritaria**    | Número de casos en clase menos frecuente | 741                       | 741                                    |
|                                  | Tamaño de la clase más pequeña           |                           |                                        |




## Qué ganamos con el modelo binario?

- El modelo binario mejora considerablemente la **capacidad predictiva general**, como se observa en las métricas de `accuracy`, `recall` y `f1-weighted`.
- Al reducir el número de clases, **simplifica la tarea de clasificación** y permite enfocar la capacidad del modelo en distinguir casos de interés práctico (ventas vs no ventas).
- Aporta una **mejor interpretación del fenómeno de conversión**, al distinguir simplemente entre clientes con y sin respuesta.


- La categoría `"con_ventas"` captura exitosamente todos los casos positivos (`recall` cercano a 1), lo cual puede ser útil en tareas de priorización o segmentación comercial. Esto significa que el modelo tiene una alta sensibilidad para detectar usuarios que efectivamente realizaron al menos una compra, minimizando los falsos negativos (FN). En contextos comerciales, esta capacidad resulta valiosa porque permite identificar con gran confianza a los usuarios que respondieron a una campaña, interactuaron con éxito con una promoción o mostraron comportamiento de conversión.

- Si bien el modelo aún falla en identificar correctamente los casos `"sin_ventas"` (precisión y recall de 0), su rendimiento sobre `"con_ventas"` revela una fuerte asimetría que puede aprovecharse operativamente. En particular, puede utilizarse como modelo de filtrado primario para identificar universos con alta probabilidad de conversión, sabiendo que el costo del error está mayormente concentrado en los usuarios no compradores, lo cual puede ser tolerable dependiendo del objetivo de negocio.
---


# Enfoque condicional sobre la categoría no explicada (sin ventas)

- **Segmentación analítica focalizada**: al aislar únicamente los casos de `"sin_ventas"` y analizar sus características internas, se pueden identificar patrones estructurales que los diferencian del resto. Esto permite construir un modelo adaptado exclusivamente a explicar la no conversión.

- **Problemas de desbalance corregidos ex post**: en el modelo original, `"sin_ventas"` es minoritaria y por tanto opacada en el proceso de aprendizaje. Al analizarla de forma independiente, se evita que el modelo la ignore.

- **Modelos especializados por subgrupo**: esta estrategia permite diseñar árboles o ensambles que capturen relaciones sutiles que se pierden en el conjunto completo. Es útil especialmente si se sospecha que el comportamiento de no compra responde a mecanismos propios (barrera de acceso, segmentación por perfil, precios, etc.).

- **Exploración de hipótesis diferenciadas**: sobre la población de `"sin_ventas"` puede testearse si existen razones estructurales o contextuales que expliquen la no conversión (por ejemplo, no logueados, navegación breve, falta de promociones visibles, etc.), algo que sería difícil de ver en un modelo general.

- **Complemento explicativo, no sustituto**: el objetivo no es reemplazar el modelo original, sino complementarlo. Se trataría de una segunda capa analítica que profundiza en la parte mal explicada del conjunto completo.


...

Esto puede implementarse fácilmente construyendo un nuevo DataFrame `df_nada = df[df['ventas_bin'] == 'sin_ventas']` y redefiniendo los modelos sobre esa subpoblación. El análisis resultante puede arrojar pistas clave sobre cómo mejorar estrategias de retención, rediseñar la interfaz de navegación o repensar la asignación de promociones. Aquí habría que arrancar nuevamente desde las regresiones, árbol y luego forest. 


### Tipos de trabajo y variantes de Random Forest para reducir overfitting y aumentar generalización

1. Bootstrapping (bagging de observaciones)

2. Submuestreo de variables por nodo (feature bagging)

3. Corte por profundidad (max_depth)

4. Tamaño mínimo de nodos (min_samples_split)

5. Número de árboles (n_estimators)

6. Bagging con corte estructurado (blocked bagging / temporal bagging)

7. Técnicas combinadas: Extremely Randomized Trees (ExtraTrees)

### Tipos de trabajo y variantes de Random Forest para reducir overfitting y aumentar generalización

El algoritmo de Random Forest combina múltiples árboles de decisión y les introduce **variación aleatoria** para generar diversidad entre ellos. Esa diversidad es clave para reducir el sobreajuste (*overfitting*). A continuación se listan y explican las principales técnicas utilizadas para "generar ruido controlado", romper correlaciones y mejorar el rendimiento del modelo.

---

##### 1. **Bootstrapping (bagging de observaciones)**  
- Cada árbol se entrena con una muestra aleatoria del dataset **con reemplazo**.  
- Algunos casos se repiten, otros no se usan (≈ 63% únicos).  
- **Objetivo**: Introducir variabilidad en los datos de entrenamiento para cada árbol.  
- Reduce la varianza del modelo final al promediar árboles distintos.  

##### 2. **Submuestreo de variables por nodo (feature bagging)**  
- Para cada *split*, se selecciona aleatoriamente un subconjunto de variables (no todas).  
- Clasificación: por defecto usa `sqrt(n_features)` variables por *split*.  
- Regresión: por defecto usa `n_features / 3`.  
- Evita que siempre se elijan las variables más fuertes, forzando diversidad entre árboles.  

##### 3. **Corte por profundidad (`max_depth`)**  
- Limita la profundidad máxima de cada árbol.  
- Árboles menos profundos → más sesgo pero menos overfitting.  
- Útil cuando hay variables correlacionadas o desbalance de clases.  

##### 4. **Tamaño mínimo de nodos (`min_samples_split`)**  
- `min_samples_split`: número mínimo de muestras para permitir un *split*.  
- Impide que el árbol aprenda reglas basadas en pocos casos (reduce ruido).  
- Valores altos generan árboles más simples y generalizables.  

##### 5. **Número de árboles (`n_estimators`)**  
- Más árboles mejoran la estabilidad del promedio o votación.  
- No causa overfitting, pero aumenta costo computacional.  
- Típicamente se usan entre 100-500 árboles (depende del dataset).  



In [ ]:

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

# Cargar dataset
df = pd.read_csv("dataset_suscripciones.csv")

# Recodificación binaria de la variable objetivo
df['ventas_bin'] = np.where(df['ventas'] == 0, 'sin_ventas', 'con_ventas')  # si ventas == 0 → "sin_ventas", si no → "con_ventas"
# Alternativa usando pd.cut:
# df['ventas_bin'] = pd.cut(df['ventas'], bins=[-1, 0, df['ventas'].max()], labels=['sin_ventas', 'con_ventas'])

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_bin']

# División fija
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42
)

# --- Random Forest con variantes explicitas (puntos 1 a 5) ---
min_leaf = int(len(X_train) * 0.02)     # Punto 4: Tamaño mínimo de nodo hoja
min_split = int(len(X_train) * 0.02)    # Punto 4: Muestras mínimas para split
max_depth = 40                          # Punto 3: Límite de profundidad del árbol
seed = 42                               

modelo_rf = RandomForestClassifier(
    n_estimators=100,                  # Punto 5: Número de árboles a considerar en el forest
    max_depth=max_depth,              # Punto 3
    min_samples_leaf=min_leaf,        # Punto 4
    min_samples_split=min_split,      # Punto 4
    random_state=seed,                
    max_features='sqrt'               # Punto 2: Feature Bagging
)                                      # Punto 1 (implícito): Bootstrapping por defecto, es lo que hace el random forest
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rec_rf = recall_score(y_test, y_pred_rf, average='weighted', zero_division=0)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

# Resultados
print("\n=== RANDOM FOREST BINARIO (sin_ventas vs con_ventas) ===")
print(f"Accuracy: {acc_rf:.4f}")
print(f"Precision (weighted): {prec_rf:.4f}")
print(f"Recall (weighted): {rec_rf:.4f}")
print(f"F1 Score (weighted): {f1_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, zero_division=0))

# Recap 

## ARBOL: Un tree predice una clase para cada OBSERVACIÓN recorriendo su estructura desde la raíz hasta una hoja

Para cada observación individual (fila del dataset), cada árbol la clasifica por su cuenta, es decir:
	•	La observa,
	•	la hace descender por sus splits,
	•	y aterriza en una hoja.
    •	Esa hoja tiene una distribución de clases (por ejemplo: [A: 10, B: 5])
    •	el árbol elige como clase final la mayoritaria en esa hoja (en este caso A).


## FOREST: 
- considerando N arboles
	•	Cada árbol predice una clase para cada observación.
	•	Se hace una votación por mayoría entre los árboles → se asigna la clase con más votos.
	•	En probabilidades, se cuentan cuántos árboles votaron por cada clase y se divide por N.
    - Entonces, la votación es por observación, no global, y depende de la clase mayoritaria en la hoja alcanzada por esa observación en cada árbol.

## Entropía
    - La impureza se usa durante el entrenamiento, no durante la predicción del árbol.
	•	Sirve para decidir los splits.
	•	No influye directamente en la votación final.
	•	Pero sí en qué tan puras serán las hojas (y por ende, qué tan confiables serán los votos que emiten).


# Votación en un Random Forest

La votación en un Random Forest se basa en el principio del *ensemble*, donde múltiples árboles de decisión (cada uno con sus errores y sesgos propios) se combinan para producir una predicción agregada más estable y precisa.

---

## ¿Cómo se construye cada árbol?

1. **Bootstraping**  
   - Se toma una muestra aleatoria con reemplazo del dataset original (de igual tamaño).  
   - Esto introduce variabilidad: cada árbol ve una combinación distinta de datos.

2. **Selección aleatoria de variables (random feature selection)**  
   - Para cada división (*split*) en cada árbol, se selecciona aleatoriamente un subconjunto de variables (por ejemplo, √n si n es el total de features).  
   - Esto evita que todos los árboles elijan siempre la misma variable dominante.

3. **Entrenamiento independiente**  
   - Cada árbol se entrena completo, sin podas ni regularizaciones cruzadas.

---

## ¿Cómo se hace la votación?

### a. Clasificación (problema discreto)

Cada árbol predice una clase. Luego:

- Se cuentan cuántos árboles predijeron cada clase.
- La clase con mayoría de votos gana (majority voting).

Ejemplo:

Árbol 1 → clase A  
Árbol 2 → clase B  
Árbol 3 → clase A  
Árbol 4 → clase A  
Árbol 5 → clase B  
Resultado → clase A (3 votos sobre 5)

---

### b. Regresión (problema continuo)

Cada árbol predice un valor numérico. Luego:

- Se calcula el promedio de todas las predicciones.

Ejemplo:

Árbol 1 → 34.2  
Árbol 2 → 36.7  
Árbol 3 → 35.0  
Resultado → (34.2 + 36.7 + 35.0) / 3 = 35.3

---

## ¿Qué se obtiene además de la predicción?

- En clasificación también pueden obtenerse probabilidades.
- Si 7 de 10 árboles votan por clase A, entonces:  
  P(A) = 0.7  
  P(B) = 0.3  
- Esto permite ordenar, rankear o calibrar las predicciones, por ejemplo para curvas ROC o scores de riesgo.

---

## Beneficio final

La combinación de:

- árboles distintos (por el bootstrap),  
- que exploran distintos espacios de variables (por el feature bagging),  
- y votan de forma agregada,  

conduce a un modelo con menor varianza, menor riesgo de sobreajuste y mayor capacidad de generalización, incluso si cada árbol individual es débil.

In [ ]:
# salidas gráficas de un random forest. 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report,
    roc_curve, auc,
    precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay
)


df = pd.read_csv("dataset_suscripciones.csv")
df['ventas_bin'] = np.where(df['ventas'] == 0, 0, 1)

X = df[['clicks', 'tiempo_browsing', 'paginas_visitadas', 'edad',
        'logueado', 'codigo_promocional', 'es_feriado', 'es_domingo',
        'precio', 'descuento_pct', 'rating_promedio']].copy()
y = df['ventas_bin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=42
)

modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=40,
    min_samples_leaf=int(len(X_train) * 0.02),
    min_samples_split=int(len(X_train) * 0.02),
    random_state=42,
    max_features='sqrt'
)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)
y_proba_rf = modelo_rf.predict_proba(X_test)[:, 1]

# 1. Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba_rf)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Random Forest')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_proba_rf)
plt.figure()
plt.plot(recall, precision, label='Curva Precisión-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precisión-Recall - Random Forest')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 3. Matriz de Confusión
cm = confusion_matrix(y_test, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión - Random Forest')
plt.grid(False)
plt.tight_layout()
plt.show()

# 4. Curva de Aprendizaje
train_sizes, train_scores, test_scores = learning_curve(
    modelo_rf, X_train, y_train, cv=5, scoring='f1_weighted',
    train_sizes=np.linspace(0.1, 1.0, 10), n_jobs=-1
)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, label='F1 en entrenamiento')
plt.plot(train_sizes, test_scores_mean, label='F1 en validación')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('F1 Score (ponderado)')
plt.title('Curva de Aprendizaje - Random Forest')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

Esta curva ROC indica que el modelo Random Forest tiene una capacidad muy limitada para discriminar entre clases. El área bajo la curva (AUC = 0.56) es apenas superior a 0.5, lo cual implica que el modelo está muy cerca del azar.

Lectura rápida:
	•	Línea diagonal gris: representa el rendimiento de un modelo completamente aleatorio (AUC = 0.5).
	•	Curva azul del modelo: muy cercana a la diagonal → el modelo no logra separar bien los positivos de los negativos.
	•	AUC = 0.56: marginalmente mejor que el azar. Técnicamente, el modelo tiene una probabilidad del 56% de rankear un positivo por encima de un negativo, lo cual es muy pobre.

Implicancia:
	•	Aunque el modelo tenga accuracy razonable, la discriminación real entre clases está fallando.
	•	La distribución de probabilidades predichas probablemente no esté bien calibrada.
	•	En problemas desbalanceados, esta métrica refleja mejor el desempeño que accuracy, porque no se ve engañada por la clase mayoritaria.